In [2]:
! which pip

/home/ubuntu/anaconda3/envs/pytorch_new/bin/pip


In [57]:
! pip install pandas bokeh blessings

     |████████████████████████████████| 10.6 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 662 kB 74.5 MB/s eta 0:00:01
  Created wheel for bokeh: filename=bokeh-2.3.1-py3-none-any.whl size=11299678 sha256=95cadd9cb516a880306a31435cead47a94805125beeb6e3c312607c355e75ad3
  Stored in directory: /root/.cache/pip/wheels/09/d1/a4/b53575bc2fa65fa5f52a29aafd6ed7b05f2ee5c11aa5a908ad
Successfully built bokeh


In [3]:
! pip install transformers

  Using cached transformers-4.5.1-py3-none-any.whl (2.1 MB)
     |████████████████████████████████| 3.3 MB 4.6 MB/s eta 0:00:01
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
     |████████████████████████████████| 733 kB 33.5 MB/s eta 0:00:01
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
     |████████████████████████████████| 303 kB 34.9 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.8 MB/s  eta 0:00:01


In [5]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.3 MB/s eta 0:00:01


In [6]:
! pip install tqdm==4.44.1 termcolor==1.1.0

In [ ]:
import json
import pandas as pd
import glob
import io
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 200)

prefix = '/Users/adivekar/workplace/coursework/nlp-qa-finalproj/results/'

def get_file_str(file_path) -> str:
    file_str: str = None
    try:
        with io.open(file_path, 'r') as inp:
            file_str = inp.read()
    except Exception as e:
        traceback.print_exc()
        return None
    return file_str
results = []

for file_path in sorted(glob.glob(prefix+'*')):
    file_txt = get_file_str(file_path)
    result = json.loads(file_txt[file_txt.find('AVERAGE') + len('AVERAGE:'):file_txt.find('MIN')].strip())
    result['EM'] = round(result['EM'], 1)
    result['F1'] = round(result['F1'], 1)
    result['file'] = file_path.replace(prefix, '')
    result['params'] = '-'.join(result['file'].split('-')[2:])
    result['dataset'] = result['file'].split('_')[0].capitalize()
    for col in ['train_dataset.size', 'train_dataset.num_paraphrased_questions', 'dev_dataset.size', 'dev_dataset.num_paraphrased_questions']:
        result[col] = int(result[col])
    if result['params'] == '' and result['train_dataset.size'] == result['train_dataset.num_paraphrased_questions']:
        result['train_dataset.num_paraphrased_questions'] = 0
    if result['params'] == '' and result['dev_dataset.size'] == result['dev_dataset.num_paraphrased_questions']:
        result['dev_dataset.num_paraphrased_questions'] = 0
    result['train_paraphrase_%'] = str(round(100 * result['train_dataset.num_paraphrased_questions'] / result['train_dataset.size'], 1)) + '%'
    results.append(result)
results = pd.DataFrame(results)[
    ['dataset', 'params', 'EM', 'F1', 'train_paraphrase_%',
    'train_dataset.num_paraphrased_questions', 'train_dataset.size', 'dev_dataset.num_paraphrased_questions', 'dev_dataset.size', 'file']
].rename(columns={
    'train_dataset.size':'train_size',
    'train_dataset.num_paraphrased_questions': 'train_num_paraphrased',
    'dev_dataset.size':'dev_size',
    'dev_dataset.num_paraphrased_questions': 'dev_num_paraphrased',
})

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
import transformers
print(transformers.__version__)

1.7.1
True
4.5.1


In [2]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
from transformers import Trainer, TrainingArguments

In [17]:
for i, model_params in enumerate([
    '--architecture pegasus --pretrained_model_name "google/pegasus-cnn_dailymail" --num_beams 75',
    '--architecture pegasus --pretrained_model_name "google/pegasus-xsum" --num_beams 75',
    '--architecture pegasus --pretrained_model_name "google/pegasus-large" --num_beams 75',
    '--architecture pegasus --pretrained_model_name "google/pegasus-multi_news" --num_beams 75',
    '--architecture fairseq --num_beams 20',
    '--architecture fairseq --MT_sampling --num_beams 20',
    ]):
    cmd = f"""
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/bioasq_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device {i} &
    """
    print(cmd, end='\n')


python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/bioasq_train.jsonl.gz" --paraphrase question --architecture pegasus --pretrained_model_name "google/pegasus-cnn_dailymail" --num_beams 75 --num_checkpoints 0 --batch_size 1 --use_gpu --device 0 &
    

python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/bioasq_train.jsonl.gz" --paraphrase question --architecture pegasus --pretrained_model_name "google/pegasus-xsum" --num_beams 75 --num_checkpoints 0 --batch_size 1 --use_gpu --device 1 &
    

python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/bioasq_train.jsonl.gz" --paraphrase question --architecture pegasus --pretrained_model_name "google/pegasus-large" --num_beams 75 --num_checkpoints 0 --batch_size 1 --use_gpu --device 2 &
    

python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/bioasq_train.jsonl.gz" --paraphrase question --arc

In [13]:
for model_params in [
    '--architecture pegasus --pretrained_model_name "google/pegasus-cnn_dailymail" --num_beams 30',
    '--architecture pegasus --pretrained_model_name "google/pegasus-xsum" --num_beams 30',
    '--architecture pegasus --pretrained_model_name "google/pegasus-large" --num_beams 30',
    '--architecture pegasus --pretrained_model_name "google/pegasus-multi_news" --num_beams 30',
    '--architecture fairseq --num_beams 20',
    '--architecture fairseq --MT_sampling --num_beams 20',
    ]:
    cmd = f"""
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 0 --examples_range 1:1500 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 1 --examples_range 1501:3000 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 2 --examples_range 3001:4500 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 3 --examples_range 4501:6000 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 4 --examples_range 6001:7500 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 5 --examples_range 7501:9000 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 6 --examples_range 9001:10500 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 7 --examples_range 10501:12000 &
    """
    print(cmd, end='\n\n\n')


python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question --architecture pegasus --pretrained_model_name "google/pegasus-cnn_dailymail" --num_beams 30 --num_checkpoints 0 --batch_size 1 --use_gpu --device 0 --examples_range 1:1500 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question --architecture pegasus --pretrained_model_name "google/pegasus-cnn_dailymail" --num_beams 30 --num_checkpoints 0 --batch_size 1 --use_gpu --device 1 --examples_range 1501:3000 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question --architecture pegasus --pretrained_model_name "google/pegasus-cnn_dailymail" --num_beams 30 --num_checkpoints 0 --batch_size 1 --use_gpu --device 2 --examples_range 3001:4500 &
python3 paraphrase.py --use_scoring_function --score_threshold 

In [21]:
import gzip, json
from tqdm import tqdm
import pandas as pd
import numpy as np
elems = None
# path = 'datasets/squad_dev.jsonl.gz'
# path = 'datasets/bioasq.jsonl.gz'
# path = 'datasets/bioasq_train.jsonl.gz'
# path = 'datasets/bioasq_dev.jsonl.gz'
# path = 'datasets/newsqa_train.jsonl.gz'
# path = 'datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz'
# path = 'datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th1.2-ngram1.jsonl.gz'
# path = 'datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th1.2-ngram1-11420.jsonl.gz'
path = "datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5.jsonl.gz"
# path = "datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz"
with gzip.open(path, 'rb') as f:
    elems = [
        json.loads(l.rstrip())
        for l in tqdm(f, desc=f'loading \'{path}\'', leave=False)
    ]

In [22]:
len(elems2)

1054

In [23]:
len(elems)

1054

In [24]:
from AbstractParaphraser import AbstractParaphraser

In [27]:
num_paraphrased_questions = 0
for elem, elem2 in zip(elems[1:], elems2[1:]):
    for qa, qa2 in zip(elem['qas'], elem2['qas']):
        print('===QA1===')
#         print(qa['question_tokens'])
#         print(qa['question_tokens_original'])
        question_tokens_original = [x[0] for x in qa['question_tokens_original']]
        question_tokens = [x[0] for x in qa['question_tokens']]
        print(f'question_tokens_original: {question_tokens_original}')
        print(f'question_tokens: {question_tokens}')
        paraphrase_score: float = AbstractParaphraser._calculate_paraphrase_score(
            question_tokens_original,
            question_tokens,
            1,
        )
        print(paraphrase_score)
        
        print('========')
        print('===QA2===')
#         print(qa2['question_tokens'])
#         print(qa2['question_tokens_original'])
        question_tokens_original = [x[0] for x in qa2['question_tokens_original']]
        question_tokens = [x[0] for x in qa2['question_tokens']]
        print(f'question_tokens_original: {question_tokens_original}')
        print(f'question_tokens: {question_tokens}')
        paraphrase_score: float = AbstractParaphraser._calculate_paraphrase_score(
            question_tokens_original,
            question_tokens,
            1,
        )
        print(paraphrase_score)
        break
    break

===QA1===
question_tokens_original: ['Which', 'is', 'the', 'major', 'RNA', 'editing', 'enzyme', 'in', 'Drosophila', 'melanogaster', '?']
question_tokens: ['Which', 'is', 'the', 'major', 'RNA', 'editing', 'enzyme', 'in', 'Drosophila', 'melanogaster']
0.909
===QA2===
question_tokens_original: ['Which', 'is', 'the', 'major', 'RNA', 'editing', 'enzyme', 'in', 'Drosophila', 'melanogaster', '?']
question_tokens: ['What', 'does', 'RNA', 'editing', 'enzyme', 'do', '?', 'What', 'is', 'the', 'difference', 'between']
1.375


In [10]:
newsqa_paraphrased_full[:10000] == elems[:10000]

True

In [3]:
newsqa_paraphrased_full = elems

In [4]:
len(newsqa_paraphrased_full)

11429

In [ ]:
for model_params in [
    '--architecture fairseq',
    '--architecture fairseq --MT_sampling --num_beams 20',
    '--architecture pegasus --pretrained_model_name "google/pegasus-cnn_dailymail"',
    '--architecture pegasus --pretrained_model_name "google/pegasus-xsum"',
    '--architecture pegasus --pretrained_model_name "google/pegasus-large"',
    '--architecture pegasus --pretrained_model_name "google/pegasus-multi_news"',
    ]:
    cmd = f"""
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 0 --examples_range 1:1500 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 1 --examples_range 1501:3000 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 2 --examples_range 3001:4500 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 3 --examples_range 4501:6000 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 4 --examples_range 6001:7500 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 5 --examples_range 7501:9000 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 6 --examples_range 9001:10500 &
python3 paraphrase.py --use_scoring_function --score_threshold 0.0 --input_path "datasets/newsqa_train.jsonl.gz" --paraphrase question {model_params} --num_checkpoints 0 --batch_size 1 --use_gpu --device 7 --examples_range 10501:12000 &
    """
print(cmd, end='\n\n\n')

In [61]:
string = 'Which is the major RNA editing enzyme in Drosophila melanogaster?'
tokens_idxs_orig = [['Which', 0],
    ['is', 6],
    ['the', 9],
    ['major', 13],
    ['RNA', 19],
    ['editing', 23],
    ['enzyme', 31],
    ['in', 38],
    ['Drosophila', 41],
    ['melanogaster', 52],
    ['?', 64]]
tokens = [x[0] for x in tokens_idxs]

In [88]:
from typing import *
def get_token_idxs(string: str, tokens: List[str]) -> List[List]:
    tokens_idxs = []
    prev_token_idx = 0
    for token in tokens:
        idx = string.find(token, prev_token_idx)
        if idx == -1:
            idx = prev_token_idx
        tokens_idxs.append([token, idx])
        prev_token_idx = idx + 1
    assert len(tokens_idxs) == len(tokens)
    return tokens_idxs

In [89]:
get_token_idxs('" What does RNA editing enzyme do? "What is the difference between', [
    '``', 
    'What',
    'does',
    'RNA',
    'editing',
    'enzyme',
    'do',
    '?',
    '``',
    'What',
    'is',
    'the',
    'difference',
    'between'])

[['``', 0],
 ['What', 2],
 ['does', 7],
 ['RNA', 12],
 ['editing', 16],
 ['enzyme', 24],
 ['do', 31],
 ['?', 33],
 ['``', 34],
 ['What', 36],
 ['is', 41],
 ['the', 44],
 ['difference', 48],
 ['between', 59]]

In [81]:
# [['What', 0],
#  ['does', 5],
#  ['RNA', 10],
#  ['editing', 14],
#  ['enzyme', 22],
#  ['do', 29],
#  ['?', 31],
#  ['What', 33],
#  ['is', 38],
#  ['the', 41],
#  ['difference', 45],
#  ['between', 56]]

[['``', 0],
 ['What', 1],
 ['does', 6],
 ['RNA', 11],
 ['editing', 15],
 ['enzyme', 23],
 ['do', 30],
 ['?', 32],
 ['``', 33],
 ['What', 35],
 ['is', 40],
 ['the', 43],
 ['difference', 47],
 ['between', 58]]

In [60]:
string.find('W', 0)

0

In [70]:
elems[1]

{'context': "Drosophila melanogaster has a single Adar gene encoding a protein related to mammalian ADAR2 that edits transcripts encoding glutamate receptor subunits. We describe the structure of the Drosophila Adar locus and use ModENCODE information to supplement published data on Adar gene transcription, and splicing. We discuss the roles of ADAR in Drosophila in terms of the two main types of RNA molecules edited and roles of ADARs as RNA-binding proteins. Site-specific RNA editing events in transcripts encoding ion channel subunits were initially found serendipitously and subsequent directed searches for editing sites and transcriptome sequencing have now led to 972 edited sites being identified in 597 transcripts. Four percent of D. melanogaster transcripts are site-specifically edited and these encode a wide range of largely membrane-associated proteins expressed particularly in CNS. Electrophysiological studies on the effects of specific RNA editing events on ion channel subuni

In [11]:
import numpy as np
np.array([np.random.randint(0,4) for _ in range(1000)]).max()

3

In [12]:
for train_file in [
    'BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram2.jsonl.gz',
    'BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th1.5-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5.jsonl.gz',
    'BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul2.0-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-large-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-large-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram2.jsonl.gz',
    'BioASQ_train-question-pegasus-large-chk20-mul1.5-beam75-temp1.5-score-th1.5-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-large-chk20-mul1.5-beam75-temp1.5.jsonl.gz',
    'BioASQ_train-question-pegasus-large-chk20-mul2.0-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-multi_news-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-multi_news-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram2.jsonl.gz',
    'BioASQ_train-question-pegasus-multi_news-chk20-mul1.5-beam75-temp1.5-score-th1.5-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-multi_news-chk20-mul1.5-beam75-temp1.5.jsonl.gz',
    'BioASQ_train-question-pegasus-multi_news-chk20-mul2.0-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-xsum-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-xsum-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram2.jsonl.gz',
    'BioASQ_train-question-pegasus-xsum-chk20-mul1.5-beam75-temp1.5-score-th1.5-ngram1.jsonl.gz',
    'BioASQ_train-question-pegasus-xsum-chk20-mul1.5-beam75-temp1.5.jsonl.gz',
    'BioASQ_train-question-pegasus-xsum-chk20-mul2.0-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'NewsQA_train-question-pegasus-xsum-chk20-mul1.5-beam30-temp1.5-score-th1.2-ngram1.jsonl.gz',
    'NewsQA_train-question-pegasus-xsum-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz',
]:
    print(f'''python3 main.py --do_train_test_eval --model "baseline" --train_path "datasets/{train_file}" --dev_path "datasets/bioasq_dev.jsonl.gz" --hidden_dim 128 --bidirectional --use_gpu --device {np.random.randint(0,4)}''')

python3 main.py --do_train_test_eval --model "baseline" --train_path "datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram1.jsonl.gz" --dev_path "datasets/bioasq_dev.jsonl.gz" --hidden_dim 128 --bidirectional --use_gpu --device 3
python3 main.py --do_train_test_eval --model "baseline" --train_path "datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th1.2-ngram2.jsonl.gz" --dev_path "datasets/bioasq_dev.jsonl.gz" --hidden_dim 128 --bidirectional --use_gpu --device 3
python3 main.py --do_train_test_eval --model "baseline" --train_path "datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th1.5-ngram1.jsonl.gz" --dev_path "datasets/bioasq_dev.jsonl.gz" --hidden_dim 128 --bidirectional --use_gpu --device 3
python3 main.py --do_train_test_eval --model "baseline" --train_path "datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5.jsonl.gz" --dev_path 

In [ ]:
word_tokenize(elem['context'])

In [208]:
elem = elems[2]
[x[0] for x in elem['context_tokens']]

['LONDON',
 ',',
 'England',
 '(',
 'CNN',
 ')',
 '--',
 'After',
 'a',
 'week',
 'when',
 'he',
 'could',
 'not',
 'be',
 'traced',
 ',',
 'Egyptian',
 'striker',
 'Amir',
 'Zaki',
 'is',
 'back',
 'at',
 'his',
 'Premier',
 'League',
 'club',
 'side',
 'Wigan',
 'Athletic',
 'in',
 'northern',
 'England',
 '.',
 'Wigan',
 'and',
 'Egypt',
 'striker',
 'Amir',
 'Zaki',
 'has',
 'mended',
 'relations',
 'with',
 'his',
 'club',
 'manager',
 '.',
 'According',
 'to',
 'Wigan',
 'manager',
 'Steve',
 'Bruce',
 'the',
 'two',
 'have',
 'patched',
 'up',
 'their',
 'differences',
 'after',
 'he',
 'launched',
 'a',
 'verbal',
 'tirade',
 'against',
 'the',
 '26-year',
 '-',
 'old',
 'striker',
 '.',
 'Zaki',
 'told',
 'Al',
 '-',
 'Hayat',
 'TV',
 'that',
 'the',
 'pair',
 '"',
 'ended',
 'up',
 'laughing',
 '"',
 'about',
 'his',
 'absence',
 '--',
 'when',
 'he',
 'failed',
 'to',
 'return',
 'from',
 'international',
 'duty',
 'and',
 'had',
 'a',
 'hamstring',
 'strain',
 'which',
 'no

In [53]:
for dataset_path in [
    'datasets/NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-MT_sampling_k50-score-th0.0-ngram1.jsonl.gz',
    'datasets/NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-score-th0.0-ngram1.jsonl.gz',
    'datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz',
    'datasets/NewsQA_train-question-pegasus-large-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz',
    'datasets/NewsQA_train-question-pegasus-multi_news-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz',
    'datasets/NewsQA_train-question-pegasus-xsum-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz',
]:
    with gzip.open(dataset_path, 'rb') as f:
        elems = [
            json.loads(l.rstrip())
            for l in tqdm(f, desc=f'loading \'{dataset_path}\'', leave=False)
        ]
        print(len(elems))
        print(elems[0])
        print(elems[1])

loading 'datasets/NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-score-th0.0-ngram1.jsonl.gz': 206it [00:00, 2050.44it/s]                  

11429
{'header': {'dataset': 'NewsQA', 'split': 'train'}}
{'context': 'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors

loading 'datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz': 209it [00:00, 2083.75it/s]

11429
{'header': {'dataset': 'NewsQA', 'split': 'train'}}
{'context': 'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors

loading 'datasets/NewsQA_train-question-pegasus-large-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz': 209it [00:00, 2088.22it/s]          

...

loading 'datasets/NewsQA_train-question-pegasus-multi_news-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz': 211it [00:00, 2104.35it/s]

11429
{'header': {'dataset': 'NewsQA', 'split': 'train'}}
{'context': 'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors

loading 'datasets/NewsQA_train-question-pegasus-xsum-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz': 205it [00:00, 2044.13it/s]        

11429
{'header': {'dataset': 'NewsQA', 'split': 'train'}}
{'context': 'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors

11429
{'header': {'dataset': 'NewsQA', 'split': 'train'}}
{'context': 'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors

In [50]:
def merge_write(prefix):
    sorted_ranges = []
    for file_path in FileSystemUtil.list_files_in_dir('datasets/', file_glob=f'{prefix}*.jsonl.gz'):
        sorted_ranges.append(
            [int(x) for x in file_path.replace('datasets/', '').replace(prefix, '').replace('.jsonl.gz', '').split('_')]
        )
    sorted_ranges = sorted(sorted_ranges, key=lambda x: x[0])
    elems = []
    header = None
    for sorted_range in sorted_ranges:
        path = f'datasets/{prefix}{sorted_range[0]}_{sorted_range[1]}.jsonl.gz'
        print(path)
        with gzip.open(path, 'rb') as f:
            vals = [
                json.loads(l.rstrip())
                for l in tqdm(f, desc=f'loading \'{path}\'', leave=False)
            ]
            header = vals[0]
            header['header'].pop('examples_range', None)
            elems.extend(vals[1:])
    file_name = prefix.replace('-range_', '')
    out_file_path = f'datasets/{file_name}.jsonl.gz'
    print(f'\nWriting {len(elems)} elements to:\n{out_file_path}')
    with gzip.open(out_file_path, 'wb') as out:
        for ex in [header] + elems:
            ## Ref: https://stackoverflow.com/a/39451012
            out.write((json.dumps(ex) + '\n').encode('utf-8'))
    print('...done\n\n')

In [51]:
for prefix in [
    'NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1-range_',
    'NewsQA_train-question-pegasus-xsum-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1-range_',
    'NewsQA_train-question-pegasus-large-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1-range_',
    'NewsQA_train-question-pegasus-multi_news-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1-range_',
    'NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-score-th0.0-ngram1-range_',
    'NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-MT_sampling_k50-score-th0.0-ngram1-range_',
]:
    merge_write(prefix)

loading 'datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1-range_1_1500.jsonl.gz': 183it [00:00, 1824.72it/s]

datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1-range_1_1500.jsonl.gz


loading 'datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1-range_1501_3000.jsonl.gz': 186it [00:00, 1850.67it/s]

datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1-range_1501_3000.jsonl.gz


loading 'datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1-range_3001_4500.jsonl.gz': 181it [00:00, 1806.32it/s] 

...

loading 'datasets/NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-MT_sampling_k50-score-th0.0-ngram1-range_9001_10500.jsonl.gz': 178it [00:00, 1768.86it/s]

datasets/NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-MT_sampling_k50-score-th0.0-ngram1-range_9001_10500.jsonl.gz


loading 'datasets/NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-MT_sampling_k50-score-th0.0-ngram1-range_10501_11428.jsonl.gz': 190it [00:00, 1892.89it/s]

datasets/NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-MT_sampling_k50-score-th0.0-ngram1-range_10501_11428.jsonl.gz



Writing 11428 elements to:
datasets/NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-MT_sampling_k50-score-th0.0-ngram1.jsonl.gz
...done


In [2]:
dev_path = '"datasets/newsqa_dev.jsonl.gz"'

device_i = 0
total = 0
for (paraphrase_score_n_gram, paraphrase_score_thresh) in [
    (1, 0.0), 
    (1, 1.0), (1, 1.2), (1, 1.5),   
    (2, 1.0), (2, 1.2), (2, 1.5),
    (3, 1.0), (3, 1.2), (3, 1.5),
]:
    for train_path in [
        'datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/NewsQA_train-question-pegasus-xsum-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/NewsQA_train-question-pegasus-large-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/NewsQA_train-question-pegasus-multi_news-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/NewsQA_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-MT_sampling_k50-score-th0.0-ngram1.jsonl.gz',
    ]:
        cmd = f'''python3 main.py --do_train_test_eval --model "baseline" --train_path {train_path} --dev_path {dev_path} --hidden_dim 128 --bidirectional --paraphrase_score_thresh {paraphrase_score_thresh} --paraphrase_score_n_gram {paraphrase_score_n_gram} --use_gpu --device {device_i} &'''
        print(cmd, end='\n\n')
        device_i = (device_i + 1) % 8
        total += 1
        if total % 16 == 0:
            print('\n\n\n')

python3 main.py --do_train_test_eval --model "baseline" --train_path datasets/NewsQA_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz --dev_path "datasets/newsqa_dev.jsonl.gz" --hidden_dim 128 --bidirectional --paraphrase_score_thresh 0.0 --paraphrase_score_n_gram 1 --use_gpu --device 0 &

python3 main.py --do_train_test_eval --model "baseline" --train_path datasets/NewsQA_train-question-pegasus-xsum-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz --dev_path "datasets/newsqa_dev.jsonl.gz" --hidden_dim 128 --bidirectional --paraphrase_score_thresh 0.0 --paraphrase_score_n_gram 1 --use_gpu --device 1 &

python3 main.py --do_train_test_eval --model "baseline" --train_path datasets/NewsQA_train-question-pegasus-large-chk20-mul1.5-beam30-temp1.5-score-th0.0-ngram1.jsonl.gz --dev_path "datasets/newsqa_dev.jsonl.gz" --hidden_dim 128 --bidirectional --paraphrase_score_thresh 0.0 --paraphrase_score_n_gram 1 --use_gpu --device 2 &

python3 main

In [1]:
dev_path = '"datasets/bioasq_dev.jsonl.gz"'

device_i = 0
total = 0
for (paraphrase_score_n_gram, paraphrase_score_thresh) in [
    (1, 0.0), 
    (1, 1.0), (1, 1.2), (1, 1.5),   
    (2, 1.0), (2, 1.2), (2, 1.5),
    (3, 1.0), (3, 1.2), (3, 1.5),
]:
    for train_path in [
        'datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/BioASQ_train-question-pegasus-xsum-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/BioASQ_train-question-pegasus-large-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/BioASQ_train-question-pegasus-multi_news-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/BioASQ_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/BioASQ_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-MT_sampling_k50-score-th0.0-ngram1.jsonl.gz',
    ]:
        cmd = f'''python3 main.py --do_train_test_eval --model "baseline" --train_path {train_path} --dev_path {dev_path} --hidden_dim 128 --bidirectional --paraphrase_score_thresh {paraphrase_score_thresh} --paraphrase_score_n_gram {paraphrase_score_n_gram} --use_gpu --device {device_i} &'''
        print(cmd, end='\n\n')
        device_i = (device_i + 1) % 8
        total += 1
        if total % 24 == 0:
            print('\n\n\n')

python3 main.py --do_train_test_eval --model "baseline" --train_path datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz --dev_path "datasets/bioasq_dev.jsonl.gz" --hidden_dim 128 --bidirectional --paraphrase_score_thresh 0.0 --paraphrase_score_n_gram 1 --use_gpu --device 0 &

python3 main.py --do_train_test_eval --model "baseline" --train_path datasets/BioASQ_train-question-pegasus-xsum-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz --dev_path "datasets/bioasq_dev.jsonl.gz" --hidden_dim 128 --bidirectional --paraphrase_score_thresh 0.0 --paraphrase_score_n_gram 1 --use_gpu --device 1 &

python3 main.py --do_train_test_eval --model "baseline" --train_path datasets/BioASQ_train-question-pegasus-large-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz --dev_path "datasets/bioasq_dev.jsonl.gz" --hidden_dim 128 --bidirectional --paraphrase_score_thresh 0.0 --paraphrase_score_n_gram 1 --use_gpu --device 2 &

python3 main

In [4]:

dev_path = '"datasets/bioasq_dev.jsonl.gz"'

device_i = 0
total = 0
for (paraphrase_score_n_gram, paraphrase_score_thresh) in [
    (1, 0.0), 
    (1, 1.0), (1, 1.2), (1, 1.5),   
    (2, 1.0), (2, 1.2), (2, 1.5),
    (3, 1.0), (3, 1.2), (3, 1.5),
]:
    for train_path in [
        'datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/BioASQ_train-question-pegasus-xsum-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/BioASQ_train-question-pegasus-large-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/BioASQ_train-question-pegasus-multi_news-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/BioASQ_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-score-th0.0-ngram1.jsonl.gz',
        'datasets/BioASQ_train-question-fairseq-chk20-mul1.5-beam20-temp1.5-MT_sampling_k50-score-th0.0-ngram1.jsonl.gz',
    ]:
        cmd = f'''python3 main.py --do_train_test_eval --shuffle_examples --model "baseline" --train_path "datasets/bioasq_train.jsonl.gz" "{train_path}" --dev_path {dev_path} --hidden_dim 128 --bidirectional --paraphrase_score_thresh {paraphrase_score_thresh} --paraphrase_score_n_gram {paraphrase_score_n_gram} --use_gpu --device {device_i} &'''
        print(cmd, end='\n\n')
        device_i = (device_i + 1) % 8
        total += 1
        if total % 24 == 0:
            print('\n\n\n')

python3 main.py --do_train_test_eval --shuffle_examples --model "baseline" --train_path "datasets/bioasq_train.jsonl.gz" "datasets/BioASQ_train-question-pegasus-cnn_dailymail-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz" --dev_path "datasets/bioasq_dev.jsonl.gz" --hidden_dim 128 --bidirectional --paraphrase_score_thresh 0.0 --paraphrase_score_n_gram 1 --use_gpu --device 0 &

python3 main.py --do_train_test_eval --shuffle_examples --model "baseline" --train_path "datasets/bioasq_train.jsonl.gz" "datasets/BioASQ_train-question-pegasus-xsum-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz" --dev_path "datasets/bioasq_dev.jsonl.gz" --hidden_dim 128 --bidirectional --paraphrase_score_thresh 0.0 --paraphrase_score_n_gram 1 --use_gpu --device 1 &

python3 main.py --do_train_test_eval --shuffle_examples --model "baseline" --train_path "datasets/bioasq_train.jsonl.gz" "datasets/BioASQ_train-question-pegasus-large-chk20-mul1.5-beam75-temp1.5-score-th0.0-ngram1.jsonl.gz" --dev_p

In [41]:
for i, (x, y) in enumerate(zip(elems[0:-1], elems[1:])):
    if x == y:
        print(i)
        print(x)
        print(y)
        print('\n\n\n')

In [40]:
[1,2,3,4,5][1:]

[2, 3, 4, 5]

In [20]:
"""A collection of utilities to augment the Python language:"""
from typing import Any, Optional, Type
from pandas.core.frame import Series as PandasSeries, DataFrame as PandasDataFrame
import numpy as np

get_default = lambda x, default, default_2=None: (default if x is None else x) if default_2 is None \
    else (default if x is None else default_2)
if_else = lambda cond, x, y: (x if cond is True else y)
is_series = lambda x: isinstance(x, PandasSeries)
is_df = lambda x: isinstance(x, PandasDataFrame)
is_int_in_floats_clothing = lambda x: isinstance(x, float) and int(x) == x


def are_any_none(*args):
    for x in args:
        if x is None:
            return True
    return False


def are_all_not_none(*args):
    return not are_any_none(*args)


def are_all_none(*args):
    for x in args:
        if x is not None:
            return False
    return True


def are_any_not_none(*args):
    return not are_all_none(*args)


def check_isinstance_or_none(x, y, err=True):
    if x is None:
        return True
    return check_isinstance(x, y, err=err)


def check_isinstance(x, y, err=True):
    if x is None and y is type(None):
        return True
    assert isinstance(y, type) or (isinstance(y, list) and np.all([isinstance(z, type) for z in y]))
    if (isinstance(y, type) and isinstance(x, y)) or (isinstance(y, list) and np.any([isinstance(x, z) for z in y])):
        return True
    if err:
        raise TypeError(
            f'Input parameter must be of type `{y.__name__}`; found type `{type(x).__name__}` with value:\n{x}')
    return False


def check_issubclass_or_none(x, y, err=True):
    if x is None:
        return True
    return check_issubclass(x, y, err=err)


def check_issubclass(x, y, err=True):
    if x is None:
        return False
    assert isinstance(x, type)
    assert isinstance(y, type) or (isinstance(y, list) and np.all([isinstance(z, type) for z in y]))
    if (isinstance(y, type) and issubclass(x, y)) or (isinstance(y, list) and np.any([issubclass(x, z) for z in y])):
        return True
    if err:
        raise TypeError(f'Input parameter must be a subclass of type {str(y)}; found type {type(x)} with value {x}')
    return False


## Ref: https://stackoverflow.com/a/13624858/4900327
class classproperty(property):
    def __get__(self, obj, objtype=None):
        return super(classproperty, self).__get__(objtype)

    def __set__(self, obj, value):
        super(classproperty, self).__set__(type(obj), value)

    def __delete__(self, obj):
        super(classproperty, self).__delete__(type(obj))


## Ref: https://stackoverflow.com/q/6760685/4900327, Method 2 base class.
## The metaclass method in the above link did not work well with multiple inheritance.
class Singleton:
    __instance = None

    def __new__(cls, *args, **kwargs):
        if not isinstance(cls.__instance, cls):
            cls.__instance = super(Singleton, cls).__new__(cls)
        return cls.__instance

    @classproperty
    def instance(cls):
        return cls.__instance


class Registry:
    _registry = {}

    def __init_subclass__(cls, **kwargs):
        """
        Register any sub class with the base class. a child class is registered as long as it is imported into environment
        """
        super().__init_subclass__(**kwargs)
        if cls in Registry.__subclasses__():
            ## The current class is a direct subclass of Registry; it is a base class.
            cls._registry = {}
        else:
            ## The current class is a subclass of a subclass of Registry: register this class as a subclass
            cls._register_subclass()

    @classmethod
    def _register_subclass(cls):
        cls._registry[cls._get_subclass_registration_key()] = cls

    @classmethod
    def _get_subclass_registration_key(cls) -> Any:
        return str(cls.__name__)

    @classmethod
    def get_subclass(cls, key: Any) -> Optional[Type]:
        return cls._registry.get(key)

from typing import Dict, List
from enum import Enum, auto
# from eps_data_transformation_lib.util.StringUtils import StringUtils


class AutoEnum(str, Enum):
    '''
    Utility class which can be subclassed to create enums using auto().
    Also provides utility methods for common enum operations.
    '''

    def _generate_next_value_(name, start, count, last_values):
        return name

    @property
    def str(self) -> str:
        return self.__str__()

    def __repr__(self):
        return self.__str__()

    def __str__(self):
        return self.name

    def __hash__(self):
        return hash(self.__class__.__name__ + self.name)

    def __eq__(self, other):
        return self is other

    def __ne__(self, other):
        return self is not other

    def matches(self, enum_value: str) -> bool:
        return self is self.from_str(enum_value, raise_error=False)

    @classmethod
    def matches_any(cls, enum_value: str) -> bool:
        return cls.from_str(enum_value, raise_error=False) is not None

    @classmethod
    def from_str(cls, enum_value: str, raise_error: bool=True):
        '''
        Performs a case-insensitive lookup of the enum value string among the members of the current AutoEnum subclass.
        :param enum_value: enum value string
        :param raise_error: whether to raise an error if the string is not found in the enum
        :return: an enum value which matches the string
        :raises: ValueError if raise_error is True and no enum value matches the string
        '''
        if isinstance(enum_value, cls):
            return enum_value
        enum_value = StringUtils.assert_not_empty_and_strip(enum_value)
        enum_values_map: Dict = {e.value.lower(): e for e in list(cls)}
        enum_obj: cls = enum_values_map.get(enum_value.lower())
        if enum_obj is None and raise_error:
            raise ValueError(f'Could not find enum with value {enum_value}.')
        return enum_obj

    @classmethod
    def convert_dict_keys_to_enum(cls, d: Dict) -> Dict:
        '''
        Converts string dict keys to the matching members of the current AutoEnum subclass.
        Leaves non-string keys untouched.
        :param d: dict to transform
        :return: dict with matching string keys transformed to enum values
        '''
        out_dict = {}
        for k, v in d.items():
            if isinstance(k, str) and cls.from_str(k, raise_error=False) is not None:
                out_dict[cls.from_str(k, raise_error=False)] = v
            else:
                out_dict[k] = v
        return out_dict

    @classmethod
    def convert_dict_keys_to_str(cls, d: Dict) -> Dict:
        '''
        Converts dict keys of the current AutoEnum subclass to the matching string key.
        Leaves other keys untouched.
        :param d: dict to transform
        :return: dict with matching keys of the current AutoEnum transformed to strings.
        '''
        out_dict = {}
        for k, v in d.items():
            if isinstance(k, cls):
                out_dict[str(k)] = v
            else:
                out_dict[k] = v
        return out_dict

    @classmethod
    def convert_dict_values_to_enum(cls, d: Dict) -> Dict:
        '''
        Converts string dict values to the matching members of the current AutoEnum subclass.
        Leaves non-string values untouched.
        :param d: dict to transform
        :return: dict with matching string values transformed to enum values
        '''
        out_dict = {}
        for k, v in d.items():
            if isinstance(v, str) and cls.from_str(v, raise_error=False) is not None:
                out_dict[k] = cls.from_str(v, raise_error=False)
            else:
                out_dict[k] = v
        return out_dict

    @classmethod
    def convert_dict_values_to_str(cls, d: Dict) -> Dict:
        '''
        Converts dict values of the current AutoEnum subclass to the matching string value.
        Leaves other values untouched.
        :param d: dict to transform
        :return: dict with matching values of the current AutoEnum transformed to strings.
        '''
        out_dict = {}
        for k, v in d.items():
            if isinstance(v, cls):
                out_dict[k] = str(v)
            else:
                out_dict[k] = v
        return out_dict

    @classmethod
    def convert_list_to_enum(cls, l: List) -> List:
        '''
        Converts string list itmes to the matching members of the current AutoEnum subclass.
        Leaves non-string items untouched.
        :param l: list to transform
        :return: list with matching string items transformed to enum values
        '''
        out_list = []
        for item in l:
            if isinstance(item, str) and cls.from_str(item, raise_error=False) is not None:
                out_list.append(cls.from_str(item, raise_error=False))
            else:
                out_list.append(item)
        return out_list

from typing import Union, List, Set, Dict, Any, Optional, Type
from abc import abstractmethod, ABC
import sys, random, numpy as np, re
from math import log10, inf
from numpy import sign
# from eps_data_transformation_lib.util.StringUtils import StringUtils
# from eps_data_transformation_lib.util.LanguageUtil import is_int_in_floats_clothing, check_isinstance_or_none, \
#     check_isinstance, check_issubclass, get_default, if_else
# from eps_data_transformation_lib.util.AutoEnum import AutoEnum


def convert_dict_to_parameters(Class, d):
    from eps_data_transformation_lib.util.Parameters import Parameters
    if Class is None:
        return d
    if d is None:
        return None
    if isinstance(d, Class):
        return d
    if isinstance(d, dict) and issubclass(Class, Parameters):
        return Class(d)
    return d


class ParameterRange(ABC):
    def __str__(self):
        return f'{self.get_class_name()}(default={str(self.default())})'

    def __repr__(self):
        return str(self)

    @classmethod
    def get_class_name(cls) -> str:
        return str(cls.__name__)  ## Will return the child class name.

    def check_is_valid(self, val, name=None):
        if not self.is_valid(val):
            raise ValueError(
                f'{get_default(val, "`None`")} is not a valid value'
                f'{if_else(StringUtils.is_not_empty(name), f" for parameter `{name}`", "")}. '
                f'{self.help_msg()}.'
            )

    @abstractmethod
    def is_valid(self, val) -> bool:
        pass

    @abstractmethod
    def default(self) -> Any:
        pass

    @abstractmethod
    def help_msg(self) -> str:
        pass


class SampleableParameterRange(ParameterRange, ABC):
    @abstractmethod
    def sample_value(self) -> Any:
        pass


class ParseableParameterRange(ParameterRange, ABC):
    @abstractmethod
    def expected_type(self) -> Type:
        pass

    @abstractmethod
    def parse_str(self, str_val) -> Any:
        pass


class IntegerRange(SampleableParameterRange, ParseableParameterRange):
    def __init__(
            self,
            low: Union[int, float],
            high: Union[int, float],
            default: Optional[Union[int, float]] = None,
            blocked_vals: List = None,
            log_sampling: bool = False,
            allow_none: bool = False,
    ):
        assert isinstance(low, int) or low == -inf or is_int_in_floats_clothing(low)
        low = int(low) if low != -inf else low
        assert isinstance(high, int) or high == inf or is_int_in_floats_clothing(high)
        high = int(high) if high != inf else high
        assert low <= high, f'Lower limit of {self.get_class_name()} ({low}) must be <= upper limit ({high})'
        assert isinstance(default, int) or is_int_in_floats_clothing(default) or default is None
        blocked_vals = get_default(blocked_vals, [])
        check_isinstance(blocked_vals, list)
        check_isinstance(log_sampling, bool)
        if log_sampling is True:
            assert low > 0, f'Lower limit of {self.get_class_name()} ({low}) must be greater than zero when using log_sampling'
        check_isinstance(allow_none, bool)
        self._low = low
        self._high = high
        self._default = int(default) if default is not None else None
        self._log_sampling = log_sampling
        self._blocked_vals = blocked_vals
        self._allow_none = allow_none

    def __str__(self):
        return f'{self.get_class_name()}(' \
               f'default={self.default()}, ' \
               f'low={self._low}, ' \
               f'high={self._high}, ' \
               f'log_sampling={self._log_sampling}, ' \
               f'blocked_vals={self._blocked_vals}, ' \
               f'allow_none={self._allow_none})'

    def expected_type(self) -> Type:
        return int

    def parse_str(self, str_val) -> Optional[int]:
        if str_val is None:
            if not self._allow_none:
                raise ValueError(f'Cannot parse None value')
            return None
        out_val = StringUtils.convert_str_to_type(str_val, int)
        self.check_is_valid(out_val)
        return out_val

    def is_valid(self, val) -> bool:
        if val is None:
            return self._allow_none
        return isinstance(val, int) and self._low <= val <= self._high and val not in self._blocked_vals

    def default(self) -> Optional[int]:
        return self._default

    def sample_value(self) -> Any:
        sampled_val = None
        ## Since numpy cannot sample with -inf and inf, we sample with the max word size,
        ## e.g. (-2^63 - 1) and (2^63 - 1). Ref: https://stackoverflow.com/a/7604981
        high = min(self._high, sys.maxsize)
        low = max(self._low, -sys.maxsize - 1)
        while sampled_val is None or sampled_val in self._blocked_vals:
            if not self._log_sampling:
                ## Samples in the interval [low, high].
                sampled_val = round(np.random.uniform(low, high))
            else:
                ## Samples exponentially in the interval [low, high]:
                sampled_val = int(round(10 ** np.random.uniform(log10(low), log10(high))))
        return sampled_val

    def help_msg(self) -> str:
        return f'The parameter value should be {self.expected_type()} ' \
               f'between {self._low} and {self._high} (inclusive). ' \
               f'The default value is: {str(self.default())}. ' \
               f'{if_else(len(self._blocked_vals) > 0, f"Values {self._blocked_vals} are not allowed. ", "")}' \
               f'`None` {if_else(self._allow_none, "can", "cannot")} be passed as an input value. ' \
               f'\nSome example values are: ' \
               f'{", ".join([str(self.sample_value()) for _ in range(min(3, self._high - self._low))])}.'


class FloatRange(SampleableParameterRange, ParseableParameterRange):
    def __init__(
            self,
            low: Union[int, float],
            high: Union[int, float],
            default: Optional[Union[int, float]] = None,
            blocked_vals: List = None,
            log_sampling: bool = False,
            allow_none: bool = False,
    ):
        check_isinstance(low, [int, float])
        low = float(low)
        check_isinstance(high, [int, float])
        high = float(high)
        assert low <= high, f'Lower limit of {self.get_class_name()} ({low}) must be <= upper limit ({high})'
        check_isinstance_or_none(default, [int, float])
        blocked_vals = get_default(blocked_vals, [])
        check_isinstance(blocked_vals, list)
        check_isinstance(log_sampling, bool)
        if log_sampling is True:
            assert low > 0, f'Lower limit of {self.get_class_name()} ({low}) must be greater than zero when using log_sampling'
        check_isinstance(allow_none, bool)
        self._low = low
        self._high = high
        self._default = float(default) if default is not None else None
        self._log_sampling = log_sampling
        self._blocked_vals = blocked_vals
        self._allow_none = allow_none

    def __str__(self):
        return f'{self.get_class_name()}(' \
               f'default={self.default()}, ' \
               f'low={self._low}, ' \
               f'high={self._high}, ' \
               f'log_sampling={self._log_sampling}, ' \
               f'blocked_vals={self._blocked_vals}, ' \
               f'allow_none={self._allow_none})'

    def expected_type(self) -> Type:
        return float

    def parse_str(self, str_val) -> Optional[float]:
        if str_val is None:
            if not self._allow_none:
                raise ValueError(f'Cannot parse None value')
            return None
        out_val = StringUtils.convert_str_to_type(str_val, float)
        self.check_is_valid(out_val)
        return out_val

    def is_valid(self, val) -> bool:
        if val is None:
            return self._allow_none
        return type(val) in [float, int] and self._low <= val <= self._high and val not in self._blocked_vals

    def default(self) -> Optional[float]:
        return self._default

    def sample_value(self) -> Any:
        sampled_val = None
        ## Since numpy cannot sample with -inf and inf, we sample with the max word size,
        ## e.g. (-2^63 - 1) and (2^63 - 1). Ref: https://stackoverflow.com/a/7604981
        high = min(self._high, sys.maxsize)
        low = max(self._low, -sys.maxsize - 1)
        while sampled_val is None or sampled_val in self._blocked_vals:
            if not self._log_sampling:
                ## Samples in the interval [low, high]:
                sampled_val = np.random.uniform(low, high)
            else:
                ## Samples exponentially in the interval [low, high]:
                sampled_val = 10 ** np.random.uniform(log10(low), log10(high))
        return sampled_val

    def help_msg(self) -> str:
        return f'The parameter value should be {self.expected_type()} ' \
               f'between {self._low} and {self._high} (inclusive). ' \
               f'\nThe default value is: {str(self.default())}. ' \
               f'{if_else(len(self._blocked_vals) > 0, f"Values {self._blocked_vals} are not allowed. ", "")}' \
               f'`None` {if_else(self._allow_none, "can", "cannot")} be passed as an input value. ' \
               f'\nSome example values are: {", ".join([str(self.sample_value()) for _ in range(3)])}.'


class CategoricalRange(SampleableParameterRange):
    def __init__(
            self,
            categories: Union[List, Set],
            default: Optional[Any] = None,
            fuzzy_match: bool = False,
            allow_none: bool = False
    ):
        assert type(categories) in [list, set] and len(categories) > 0
        assert default in categories or default is None
        check_isinstance(fuzzy_match, bool)
        check_isinstance(allow_none, bool)
        self._categories = list(categories)
        self._categories_str_uncased = [str(category).strip().lower() for category in self._categories]
        self._default = default
        self._fuzzy_match = fuzzy_match
        self._allow_none = allow_none

    def __str__(self):
        return f'{self.get_class_name()}(' \
               f'default={self.default()}, ' \
               f'categories={self._categories}, ' \
               f'default={self._default}, ' \
               f'fuzzy_match={self._fuzzy_match}, ' \
               f'allow_none={self._allow_none})'

    def is_valid(self, val) -> bool:
        if val is None:
            return self._allow_none
        if not self._fuzzy_match:
            return val in self._categories
        else:
            return str(val).strip().lower() in self._categories_str_uncased

    def default(self) -> Any:
        return self._default

    def sample_value(self) -> Any:
        return np.random.choice(self._categories)

    def help_msg(self) -> str:
        return f'The parameter value should be one of {self._categories} ' \
               f'{if_else(self._fuzzy_match, f"(matched fuzzily) ", "")}' \
               f'\nThe default value is: {str(self.default())}. ' \
               f'`None` {if_else(self._allow_none, "can", "cannot")} be passed as an input value. ' \
               f'\nSome example values are: {", ".join([str(self.sample_value()) for _ in range(3)])}.'


class BoolRange(CategoricalRange, ParseableParameterRange):
    def __init__(self, *args, **kwargs):
        super().__init__([True, False, 0, 1], *args, **kwargs)

    def expected_type(self) -> Type:
        return bool

    def parse_str(self, str_val) -> Optional[bool]:
        if str_val is None:
            if not self._allow_none:
                raise ValueError(f'Cannot parse None value')
            return None
        out_val = StringUtils.convert_str_to_type(str_val, bool)
        self.check_is_valid(out_val)
        return out_val

    def help_msg(self) -> str:
        return f'The parameter value should be {self.expected_type()} ' \
               f'{if_else(self._fuzzy_match, f"(matched fuzzily). ", ". ")}' \
               f'\nThe default value is: {str(self.default())}. ' \
               f'`None` {if_else(self._allow_none, "can", "cannot")} be passed as an input value. ' \
               f'\nSupported values are: {", ".join(self._categories)}.'


class StringRange(SampleableParameterRange, ParseableParameterRange):
    def __init__(
            self,
            min_len: int,
            max_len: [int, float],
            default: Optional[str] = None,
            blocked_vals: List = None,
            allow_none: bool = False,
            regex: str = None,
            max_sample_length: int = 50,
    ):
        assert isinstance(min_len, int) and min_len >= 0
        assert (isinstance(max_len, int) or max_len == inf) and max_len >= 1
        assert isinstance(max_sample_length, int) and max_sample_length >= 0
        max_sample_length = if_else(max_sample_length > max_len, max_len, max_sample_length)
        check_isinstance_or_none(default, str)
        check_isinstance_or_none(blocked_vals, list)
        check_isinstance(allow_none, bool)
        blocked_vals = get_default(blocked_vals, [])
        check_isinstance_or_none(regex, str)
        regex: str = get_default(regex, StringUtils.MATCH_ALL_REGEX_SINGLE_LINE)
        self._min_len = min_len
        self._max_len = max_len
        self._max_sample_length = max_sample_length
        self._default = default
        self._blocked_vals = blocked_vals
        self._regex = regex
        self._allow_none = allow_none

    def __str__(self):
        return f'{self.get_class_name()}(' \
               f'default={self.default()}, ' \
               f'min_len={self._min_len}, ' \
               f'max_len={self._max_len}, ' \
               f'max_sample_length={self._max_sample_length}, ' \
               f'blocked_vals={self._blocked_vals}, ' \
               f'regex={self._regex}, ' \
               f'allow_none={self._allow_none})'

    def expected_type(self) -> Type:
        return str

    def parse_str(self, str_val) -> Optional[str]:
        if str_val is None:
            if not self._allow_none:
                raise ValueError(f'Cannot parse None value')
            return None
        out_val = StringUtils.convert_str_to_type(str_val, str)
        self.check_is_valid(out_val)
        return out_val

    def is_valid(self, val) -> bool:
        if val is None:
            return self._allow_none
        if not isinstance(val, str):
            return False
        if not (self._min_len <= len(val) <= self._max_len):
            return False
        if val in self._blocked_vals:
            return False
        if re.compile(self._regex).match(val) is None:
            return False
        return True

    def default(self) -> str:
        return self._default

    def sample_value(self) -> Any:
        min_len = self._min_len
        max_len = self._max_sample_length
        sampled_val = None
        while sampled_val is None or sampled_val in self._blocked_vals:
            sampled_val = StringUtils.generate_random_strings(min_num_chars=min_len, max_num_chars=max_len)
        return sampled_val

    def help_msg(self) -> str:
        return f'The parameter value should be {self.expected_type()} ' \
               f'between {self._min_len} and {self._max_len} characters (inclusive). ' \
               f'It should match regex: {self._regex}' \
               f'\nThe default value is: {str(self.default())}. ' \
               f'{if_else(len(self._blocked_vals) > 0, f"Values {self._blocked_vals} are not allowed. ", "")}' \
               f'`None` {if_else(self._allow_none, "can", "cannot")} be passed as an input value. ' \
               f'\nSome example values are:\n' + "\n".join([f'{i + 1}. {str(self.sample_value())}' for i in range(3)])


class AutoEnumValue(SampleableParameterRange, ParseableParameterRange):
    def __init__(
            self,
            ParamEnum: AutoEnum.__class__,
            default: Optional[AutoEnum] = None,
            allow_none: bool = False,
    ):
        check_isinstance(ParamEnum, AutoEnum.__class__)
        check_isinstance_or_none(default, [str, ParamEnum])
        check_isinstance(allow_none, bool)
        self._ParamEnum: AutoEnum.__class__ = ParamEnum
        if default is None:
            self._default: AutoEnum = None
        else:
            self._default: AutoEnum = ParamEnum.from_str(default, raise_error=False)
        self._allow_none: bool = allow_none

    def expected_type(self) -> Type:
        return self._ParamEnum

    def parse_str(self, str_val) -> Optional[AutoEnum]:
        if str_val is None:
            if not self._allow_none:
                raise ValueError(f'Cannot parse None value')
            return None
        out_val = self._ParamEnum.from_str(str_val)
        self.check_is_valid(out_val)
        return out_val

    def __str__(self):
        return f'{self.get_class_name()}(' \
               f'ParamEnum=`{self._ParamEnum.__name__}`, ' \
               f'default={self.default()}, ' \
               f'allow_none={self._allow_none})'

    def is_valid(self, val) -> bool:
        if val is None:
            return self._allow_none
        try:
            return isinstance(self._ParamEnum.from_str(val, raise_error=False), self._ParamEnum)
        except Exception as e:
            return False

    def default(self) -> str:
        return self._default

    def sample_value(self) -> AutoEnum:
        return list(self._ParamEnum)[np.random.choice(np.arange(0, len(list(self._ParamEnum))))]

    def help_msg(self) -> str:
        return f'The parameter value should be {str(self.expected_type())} ' \
               f'or a string in {[str(e.name) for e in list(self._ParamEnum)]} (not case-sensitive). ' \
               f'\nThe default value is: {str(self.default())}. ' \
               f'`None` {if_else(self._allow_none, "can", "cannot")} be passed as an input value. ' \
               f'\nSome example values are: ' \
               f'{", ".join([str(self.sample_value().name) for _ in range(min(3, len(list(self._ParamEnum))))])}.'


class FixedValue(ParameterRange):
    def __init__(self, val: Optional[Any]):
        self._val = val

    def is_valid(self, val) -> bool:
        return self._val == val

    def default(self) -> Any:
        return self._val

    def help_msg(self) -> str:
        return f'The only valid value is {self._val}'


class InstanceOf(ParseableParameterRange):
    def __init__(
            self,
            Classes: Union[Type, List[Type]],
            default: Optional[Type] = None,
            allow_none: bool = False,
            convert_params_dict: bool = True
    ):
        assert isinstance(Classes, type) or \
               (isinstance(Classes, list) and np.all([isinstance(x, type) for x in Classes]))
        self._Classes = if_else(isinstance(Classes, type), [Classes], Classes)
        check_isinstance(self._Classes, list)
        for class_ in self._Classes:
            check_isinstance(class_, type)
        check_isinstance_or_none(default, self._Classes)
        self._default = default
        check_isinstance(allow_none, bool)
        self._allow_none: bool = allow_none
        check_isinstance(convert_params_dict, bool)
        self._convert_params_dict: bool = convert_params_dict

    def __str__(self):
        return f'{self.get_class_name()}(' \
               f'default={self.default()}, ' \
               f'classes=[{", ".join([f"`{c.__name__}`" for c in self._Classes])}], ' \
               f'allow_none={self._allow_none})'

    def is_valid(self, val) -> bool:
        if val is None:
            return self._allow_none
        return check_isinstance(val, self._Classes, err=False)

    def default(self) -> Any:
        return self._default

    def expected_type(self) -> List[Type]:
        return self._Classes

    def parse_str(self, str_val) -> Optional[Union[Dict, 'Parameters']]:
        if str_val is None:
            if not self._allow_none:
                raise ValueError(f'Cannot parse None value')
            return None
        if len(self._Classes) == 1 and self._convert_params_dict:
            str_val = convert_dict_to_parameters(self._Classes[0], str_val)
        elif len(self._Classes) == 1 and issubclass(self._Classes[0], AutoEnum):
            str_val = self._Classes[0].from_str(str_val)
        return str_val

    def help_msg(self) -> str:
        if len(self._Classes) == 1:
            classes_str = f'`{self._Classes[0].__name__}`'
        else:
            classes_str = f'one of the following: [{", ".join([f"`{c.__name__}`" for c in self._Classes])}]'
        return f'The parameter value should be an instance of {classes_str}. ' \
               f'The default value is: {str(self.default())}. ' \
               f'`None` {if_else(self._allow_none, "can", "cannot")} be passed as an input value.'


class SubclassOf(ParameterRange):
    def __init__(
            self,
            Classes: Union[Type, List[Type]],
            default: Optional[List] = None,
            allow_none: bool = False
    ):
        assert isinstance(Classes, type) or \
               (isinstance(Classes, list) and np.all([isinstance(x, type) for x in Classes]))
        self._Classes = if_else(isinstance(Classes, type), [Classes], Classes)
        check_isinstance(self._Classes, list)
        for class_ in self._Classes:
            check_isinstance(class_, type)
        check_isinstance_or_none(default, self._Classes)
        self._default = default
        check_isinstance(allow_none, bool)
        self._allow_none: bool = allow_none

    def __str__(self):
        return f'{self.get_class_name()}(' \
               f'default={self.default()}, ' \
               f'classes=[{", ".join([f"`{c.__name__}`" for c in self._Classes])}], ' \
               f'allow_none={self._allow_none})'

    def is_valid(self, val) -> bool:
        if val is None:
            return self._allow_none
        return check_issubclass(val, self._Classes, err=False)

    def default(self) -> Any:
        return self._default

    def help_msg(self) -> str:
        return f'The parameter value should be a subclass of one of [{", ".join([f"`{c.__name__}`" for c in self._Classes])}]. ' \
               f'The default value is: {str(self.default())}. ' \
               f'`None` {if_else(self._allow_none, "can", "cannot")} be passed as an input value.'


class CollectionOf(ParameterRange, ABC):
    pass


class ListOf(CollectionOf, ParseableParameterRange):
    def __init__(
            self,
            Class: Optional[Type] = None,
            min_length: int = 0,
            max_length: Union[int, float] = inf,
            default: Optional[List] = None,
            allow_none: bool = False,
            convert_params_dict: bool = True
    ):
        check_isinstance_or_none(Class, type)
        check_isinstance(min_length, int)
        assert isinstance(max_length, int) or max_length == inf
        check_isinstance_or_none(default, list)
        check_isinstance(allow_none, bool)
        self._Class = Class
        if self._Class is not None and default is not None:
            assert np.all([check_isinstance(x, self._Class) for x in default])
        self._min_length: int = min_length
        self._max_length: Union[int, float] = max_length
        self._default = default
        self._allow_none: bool = allow_none
        check_isinstance(convert_params_dict, bool)
        self._convert_params_dict: bool = convert_params_dict

    def __str__(self):
        return f'{self.get_class_name()}(' \
               f'min_length={self._min_length}, ' \
               f'max_length={self._max_length}, ' \
               f'default={self.default()}, ' \
               f'class={f"`{self._Class.__name__}`" if self._Class is not None else None}, ' \
               f'allow_none={self._allow_none})'

    def expected_type(self) -> Type:
        return list

    def parse_str(self, str_val) -> Optional[List]:
        if str_val is None:
            if not self._allow_none:
                raise ValueError(f'Cannot parse None value')
            return None
        out_val = StringUtils.convert_str_to_type(str_val, list)
        if check_issubclass(self._Class, AutoEnum, err=False):
            out_val = self._Class.convert_list_to_enum(out_val)
        if self._convert_params_dict:
            out_val = [convert_dict_to_parameters(self._Class, v) for v in out_val]
        return out_val

    def is_valid(self, val) -> bool:
        if val is None:
            return self._allow_none
        return isinstance(val, list) and (self._Class is None or np.all([isinstance(x, self._Class) for x in val]))

    def default(self) -> Any:
        return self._default

    def help_msg(self) -> str:
        return f'The parameter value should be {self.expected_type()}' \
               f'{f" where each element has type `{self._Class.__name__}`. " if self._Class is not None else ". "}' \
               f'The length should be between {self._min_length} and {self._max_length} (inclusive). ' \
               f'The default value is: {str(self.default())}. ' \
               f'`None` {if_else(self._allow_none, "can", "cannot")} be passed as an input value.'


class DictOf(CollectionOf, ParseableParameterRange):
    def __init__(
            self,
            key_class: Optional[Type] = None,
            value_class: Optional[Type] = None,
            default: Optional[Dict] = None,
            allow_none: bool = False,
            convert_params_dict: bool = True
    ):
        check_isinstance_or_none(key_class, type)
        check_isinstance_or_none(value_class, type)
        check_isinstance_or_none(default, dict)
        check_isinstance(allow_none, bool)
        check_isinstance(convert_params_dict, bool)
        self._key_class = key_class
        self._value_class = value_class
        if self._key_class is not None and default is not None:
            assert np.all([check_isinstance(k, self._key_class) for k in default.keys()])
        if self._value_class is not None and default is not None:
            assert np.all([check_isinstance(v, self._value_class) for v in default.values()])
        self._default = default
        self._allow_none: bool = allow_none
        self._convert_params_dict = convert_params_dict

    def __str__(self):
        return f'{self.get_class_name()}(' \
               f'default={self.default()}, ' \
               f'key_class={f"`{self._key_class.__name__}`" if self._key_class is not None else None}, ' \
               f'value_class={f"`{self._value_class.__name__}`" if self._value_class is not None else None}, ' \
               f'allow_none={self._allow_none})'

    def expected_type(self) -> Type:
        return dict

    def parse_str(self, str_val) -> Optional[Dict]:
        if str_val is None:
            if not self._allow_none:
                raise ValueError(f'Cannot parse None value')
            return None
        out_val = StringUtils.convert_str_to_type(str_val, dict)
        if check_issubclass(self._key_class, AutoEnum, err=False):
            out_val = self._key_class.convert_dict_keys_to_enum(out_val)
        if check_issubclass(self._value_class, AutoEnum, err=False):
            out_val = self._value_class.convert_dict_values_to_enum(out_val)
        if self._convert_params_dict:
            out_val = {k: convert_dict_to_parameters(self._value_class, v) for k, v in out_val.items()}
        return out_val

    def is_valid(self, val) -> bool:
        if val is None:
            return self._allow_none
        return isinstance(val, dict) and \
               (self._key_class is None or np.all([isinstance(k, self._key_class) for k in val.keys()])) and \
               (self._value_class is None or np.all([isinstance(k, self._value_class) for k in val.values()]))

    def default(self) -> Any:
        return self._default

    def help_msg(self) -> str:
        key_msg = if_else(self._key_class is not None, f', key has type {self._key_class}', '')
        value_msg = if_else(self._value_class is not None, f', value has type {self._value_class}', '')
        entry_msg = if_else(
            self._key_class is not None or self._value_class is not None,
            f' where, for each entry{key_msg}{value_msg}',
            ''
        )
        return f'The parameter value should be {self.expected_type()}' \
               f'{entry_msg}. ' \
               f'The default value is: {str(self.default())}. ' \
               f'`None` {if_else(self._allow_none, "can", "cannot")} be passed as an input value.'


PARAM_RANGES_DICT = DictOf(key_class=AutoEnum, value_class=ParameterRange)
LIST_OF_STRINGS = ListOf(str)

from typing import Dict, Any, List, Union, Tuple, Set
import re, random
import pandas as pd
import numpy as np
from pandas.core.frame import Series as PandasSeries, DataFrame as PandasDataFrame
# from eps_data_transformation_lib.util.LanguageUtil import get_default
# from eps_data_transformation_lib.util.AutoEnum import AutoEnum
# from eps_data_transformation_lib.util.ParameterRange import ParameterRange, PARAM_RANGES_DICT
# from eps_data_transformation_lib.util.ParameterRange import LIST_OF_STRINGS
from ast import literal_eval

ListOrTuple = Union[List, Tuple]
DataFrameOrSeries = Union[PandasSeries, PandasDataFrame]
SeriesOrArray1D = Union[PandasSeries, List, Tuple, np.ndarray]
DataFrameOrArray2D = Union[PandasSeries, PandasDataFrame, List, List[List], np.ndarray]
SeriesOrArray1DOrDataFrameOrArray2D = Union[SeriesOrArray1D, DataFrameOrArray2D]


class CollectionsUtils:
    """Utility class with static methods to process various Python collections (List, Set, Dict, etc)"""

    def __init__(self):
        raise TypeError("Cannot create 'CollectionsUtils' instances.")

    @staticmethod
    def transform_keys_case(d: Dict, case='lower'):
        """
        Converts string dict keys to either uppercase or lowercase. Leaves non-string keys untouched.
        :param d: dict to transform
        :param case: desired case, either 'lower' or 'upper'
        :return: dict with case-transformed keys
        """
        assert isinstance(case, str)
        case = case.lower()
        if case not in ['lower', 'upper']:
            raise AttributeError(f'Unknown case {case}')
        out = {}
        for k, v in d.items():
            if isinstance(k, str):
                if case == 'lower':
                    out[k.lower()] = v
                elif case == 'upper':
                    out[k.upper()] = v
            else:
                out[k] = v
        return out

    @staticmethod
    def transform_values_case(d: Dict, case='lower'):
        """
        Converts string dict values to either uppercase or lowercase. Leaves non-string values untouched.
        :param d: dict to transform
        :param case: desired case, either 'lower' or 'upper'
        :return: dict with case-transformed values
        """
        assert isinstance(case, str)
        case = case.lower()
        if case not in ['lower', 'upper']:
            raise AttributeError(f'Unknown case {case}')
        out = {}
        for k, v in d.items():
            if isinstance(v, str):
                if case == 'lower':
                    out[k] = v.lower()
                elif case == 'upper':
                    out[k] = v.upper()
            else:
                out[k] = v
        return out

    @staticmethod
    def dict_convert_keys_to_enum(
            d: Dict,
            EnumClass: AutoEnum.__class__,
            nested_EnumClass_mapping: Dict[AutoEnum, AutoEnum.__class__] = None,
            err: bool = False,
    ):
        """
        Recursively, converts string dict keys to the matching enum values of the AutoEnum class passed.
        Leaves non-string keys untouched.
        :param d: dict to transform
        :param EnumClass: the AutoEnum class from which string keys will be matched to enum values
        :return: dict with matching string keys transformed to enum values
        """
        if d is None:
            d = {}
        assert isinstance(d, dict)
        if EnumClass is None:
            return d
        assert isinstance(EnumClass, AutoEnum.__class__)
        d = EnumClass.convert_dict_keys_to_enum(d)
        if CollectionsUtils.is_empty_dict(nested_EnumClass_mapping):
            return d
        assert isinstance(nested_EnumClass_mapping, dict)
        out_d = {}
        for k, v in d.items():
            if err and k not in EnumClass:
                raise KeyError(f'Unrecognized param name "{k}": valid params names are {list(EnumClass)}.')
            if k in nested_EnumClass_mapping and isinstance(v, dict):
                assert isinstance(k, AutoEnum), f'Nested key {str(k)} should be as SomeEnum.ENUM_VAL, not SomeEnum'
                nested_main_EnumClass = nested_EnumClass_mapping[k]
                assert isinstance(nested_main_EnumClass, AutoEnum.__class__)
                out_d[k]: Dict = CollectionsUtils.dict_convert_keys_to_enum(
                    v, EnumClass=nested_main_EnumClass, nested_EnumClass_mapping=nested_EnumClass_mapping
                )
            else:
                out_d[k] = v
        return out_d

    @staticmethod
    def dict_set_default(d: Dict, default_params: Dict) -> Dict:
        """
        Sets default values in a dict for missing keys
        :param d: input dict
        :param default_params: dict of default values
        :return: input dict with default values populated for missing keys
        """
        if d is None:
            d = {}
        assert isinstance(d, dict)
        if default_params is None:
            return d
        assert isinstance(default_params, dict)
        for k, v in default_params.items():
            if isinstance(v, dict) and isinstance(d.get(k), dict):
                ## We need to go deeper:
                d[k] = CollectionsUtils.dict_set_default(d[k], v)
            else:
                d.setdefault(k, v)
        return d

    @staticmethod
    def filter_string_list(l: List[str], pattern: str, ignorecase=False) -> List[str]:
        """
        Filter a list of strings based on an exact match to a regex pattern. Leaves non-string items untouched.
        :param l: list of strings
        :param pattern: Regex pattern used to match each item in list of strings.
        Strings which are not a regex pattern will be expected to exactly match.
        E.g. the pattern 'abcd' will only match the string 'abcd'.
        To match 'abcdef', pattern 'abcd.*' should be used.
        To match 'xyzabcd', patterm '.*abcd' should be used.
        To match 'abcdef', 'xyzabcd' and 'xyzabcdef', patterm '.*abcd.*' should be used.
        :param ignorecase: whether to ignore case while matching the pattern to the strings.
        :return: filtered list of strings which match the pattern.
        """
        if not pattern.startswith('^'):
            pattern = '^' + pattern
        if not pattern.endswith('$'):
            pattern = pattern + '$'
        flags = 0
        if ignorecase:
            flags = flags | re.IGNORECASE
        return [x for x in l if not isinstance(x, str) or len(re.findall(pattern, x, flags=flags)) > 0]

    @staticmethod
    def filter_list_values(l: List[str], values: Any, exclude: bool = False) -> List[str]:
        """
        Filter a list of items (of any type) to a list of values.
        :param l: list of items
        :param values: list of values
        :param exclude: how to use the list of filter values.
            - exclude=False, values acts as a whitelist (Default)
            - exclude=True, values acts as a blacklist
        :return: filtered list of items
        """
        if not isinstance(values, list):
            values = [values]
        if exclude:
            return [x for x in l if x not in values]
        return [x for x in l if x in values]

    @staticmethod
    def filter_dict_keys_on_list(d: Dict, filtered_keys: List, exclude: bool = False) -> Dict:
        """
        Filter values in a dict based on a list of keys.
        :param d: dict to filter
        :param filtered_keys: list of keys to include/exclude.
        :param exclude: whether to keep or remove keys in filtered_keys list.
        :return: dict with filtered list of keys
        """
        if exclude:
            return {k: d[k] for k in d if k not in filtered_keys}
        return {k: d[k] for k in filtered_keys if k in d}

    @staticmethod
    def convert_and_filter_dict_keys_on_enum(d: Dict, AutoEnumClass: AutoEnum.__class__, exclude: bool = False) -> Dict:
        """
        Filter values in a dict based on those matching an enum.
        :param d: dict to filter
        :param AutoEnumClass: AutoEnum class on which to filter
        :param exclude: whether to keep or remove keys in the AutoEnum class
        :return: dict with filtered list of keys
        """
        if AutoEnumClass is None:
            return {}
        assert isinstance(AutoEnumClass, AutoEnum.__class__)
        d = CollectionsUtils.dict_convert_keys_to_enum(d, AutoEnumClass)
        return CollectionsUtils.filter_dict_keys_on_list(d, list(AutoEnumClass), exclude=exclude)

    @staticmethod
    def filter_dict_keys_on_pattern(d: Dict, key_pattern: str, ignorecase=False):
        """
        Filter string keys in a dict based on a regex pattern.
        :param d: dict to filter
        :param key_pattern: regex pattern used to match keys.
        Follows same rules as `filter_string_list` method, i.e. only checks string keys and retains non-string keys.
        :return: dict with filtered keys
        """
        keys: List = list(d.keys())
        filtered_keys: List = CollectionsUtils.filter_string_list(keys, key_pattern, ignorecase=ignorecase)
        return CollectionsUtils.filter_dict_keys_on_list(d, filtered_keys)

    @staticmethod
    def filter_dict_values(d: Dict, values: Any, exclude: bool = False) -> Dict:
        """
        Filter values in a dict (of any type) to a list of values.
        :param d: dict to filter
        :param values: list of values (of any type)
        :param exclude: how to use the list of filter values.
            - exclude=False, keep any value in `values`
            - exclude=True, remove any value in `values`
        :return: dict with filtered values
        """
        if not isinstance(values, list):
            values = [values]
        if exclude:
            return {k: v for k, v in d.items() if v not in values}
        return {k: v for k, v in d.items() if v in values}

    @staticmethod
    def filter_dict_none_values(d: Dict, exclude: bool = True) -> Dict:
        return CollectionsUtils.filter_dict_values(d, None, exclude=exclude)

    @staticmethod
    def is_not_empty_list(l: List) -> bool:
        return isinstance(l, list) and len(l) > 0

    @staticmethod
    def is_empty_list(l: List) -> bool:
        return not CollectionsUtils.is_not_empty_list(l)

    @staticmethod
    def get_subset(small_list: ListOrTuple, big_list: ListOrTuple):
        assert CollectionsUtils.is_list_like(small_list)
        assert CollectionsUtils.is_list_like(big_list)
        return set.intersection(set(small_list), set(big_list))

    @staticmethod
    def is_subset(small_list: ListOrTuple, big_list: ListOrTuple) -> bool:
        return len(CollectionsUtils.get_subset(small_list, big_list)) == len(small_list)

    @staticmethod
    def assert_not_empty_list(l: List):
        assert CollectionsUtils.is_not_empty_list(l)

    @staticmethod
    def is_list_like(l: Union[List, Tuple]):
        return isinstance(l, list) or isinstance(l, tuple) or isinstance(l, np.ndarray)

    @staticmethod
    def is_not_list_like(l: Union[List, Tuple]):
        return not CollectionsUtils.is_list_like(l)

    @staticmethod
    def is_not_empty_list_like(l: ListOrTuple) -> bool:
        return CollectionsUtils.is_list_like(l) and len(l) > 0

    @staticmethod
    def is_empty_list_like(l: ListOrTuple) -> bool:
        return not CollectionsUtils.is_not_empty_list_like(l)

    @staticmethod
    def assert_not_empty_list_like(l: ListOrTuple, error_message=''):
        assert CollectionsUtils.is_not_empty_list_like(l), error_message

    @staticmethod
    def element_to_list(l):
        if CollectionsUtils.is_not_list_like(l):
            return [l]
        return l

    @staticmethod
    def is_not_empty_dict(d: Dict) -> bool:
        return isinstance(d, dict) and len(d) > 0

    @staticmethod
    def is_empty_dict(d: Dict) -> bool:
        return not CollectionsUtils.is_not_empty_dict(d)

    @staticmethod
    def assert_not_empty_dict(d: Dict):
        assert CollectionsUtils.is_not_empty_dict(d)

    @staticmethod
    def get_random_dict_key(d: Dict) -> Any:
        if CollectionsUtils.is_not_empty_dict(d):
            return random.choice(list(d.keys()))
        return None

    @staticmethod
    def get_random_value(struct: Union[ListOrTuple, Dict, str]) -> Any:
        if CollectionsUtils.is_not_empty_list_like(struct) or isinstance(struct, str):
            return random.choice(struct)
        elif CollectionsUtils.is_not_empty_dict(struct):
            return struct[CollectionsUtils.get_random_dict_key(struct)]
        return None

    @staticmethod
    def is_1d_array(l: Union[List, Tuple]):
        return CollectionsUtils.is_list_like(l) and len(l) > 0 and not CollectionsUtils.is_list_like(l[0])

    @staticmethod
    def is_2d_array(l: Union[List, Tuple]):
        return CollectionsUtils.is_list_like(l) and len(l) > 0 and CollectionsUtils.is_list_like(l[0])

    @classmethod
    def convert_1d_or_2d_array_to_dataframe(cls, data: SeriesOrArray1DOrDataFrameOrArray2D) -> PandasDataFrame:
        if CollectionsUtils.is_1d_array(data):
            data: PandasSeries = cls.convert_1d_array_to_series(data)
        if isinstance(data, PandasSeries) or CollectionsUtils.is_2d_array(data):
            data: PandasDataFrame = pd.DataFrame(data)
        assert isinstance(data, PandasDataFrame)
        return data

    @classmethod
    def convert_1d_array_to_series(cls, data: SeriesOrArray1D):
        if len(data) == 0:
            raise ValueError(f'Cannot convert empty data structure to series')
        if isinstance(data, PandasSeries):
            return data
        if not CollectionsUtils.is_list_like(data):
            raise ValueError(f'Cannot convert non list-like data structure to series')
        return pd.Series(data)

    @classmethod
    def get_unique_values_list(
            cls, data: SeriesOrArray1DOrDataFrameOrArray2D, exclude_nans: bool = True
    ) -> List[Any]:
        if data is None:
            return []
        if isinstance(data, PandasSeries) or isinstance(data, PandasDataFrame):
            data: np.ndarray = data.values
        if CollectionsUtils.is_2d_array(data):
            data: np.ndarray = CollectionsUtils.convert_1d_or_2d_array_to_dataframe(data).values
        if not isinstance(data, np.ndarray):
            data: np.ndarray = np.array(data)
        flattened_data = data.ravel('K')  ## 1-D array of all data (w/ nans). Ref: https://stackoverflow.com/a/26977495
        if len(flattened_data) == 0:
            return []
        if exclude_nans:
            flattened_data = flattened_data[~pd.isnull(flattened_data)]
        flattened_data = np.unique(flattened_data)
        return sorted(list(flattened_data))

    @classmethod
    def get_num_non_null_columns_per_row(cls, df: PandasDataFrame) -> PandasSeries:
        ## Ref: https://datascience.stackexchange.com/a/16801/35826
        assert isinstance(df, PandasDataFrame)
        return (~df.isna()).sum(axis=1)

    @classmethod
    def get_max_num_non_null_columns_per_row(cls, df: PandasDataFrame) -> int:
        assert isinstance(df, PandasDataFrame)
        return cls.get_num_non_null_columns_per_row(df).max()

    @classmethod
    def eval_dict_values(cls, params: Dict):
        if not isinstance(params, dict):
            raise ValueError(f"{params} should be of type dict")
        updated_dict = {}
        for parameter, value in params.items():
            try:
                updated_dict[parameter] = literal_eval(value)
            except:
                updated_dict[parameter] = value
        return updated_dict

    @classmethod
    def assert_parameter_ranges(cls, params: Dict = None, param_ranges: Dict = None):
        """Checks whether the parameters are in the required ranges."""
        if params is None or param_ranges is None:
            return True
        if not isinstance(params, dict):
            raise ValueError(f"{params} should be of type dict")
        if not isinstance(param_ranges, dict):
            raise ValueError(f"{param_ranges} should be of type dict")
        if not PARAM_RANGES_DICT.is_valid(param_ranges):
            raise ValueError(f'Invalid dict of param ranges: {param_ranges}')
        keys_to_check = set(params.keys()).intersection(set(param_ranges.keys()))
        for param_name in keys_to_check:
            param_range: ParameterRange = param_ranges[param_name]
            param_value = params[param_name]
            assert isinstance(param_range, ParameterRange), f'{param_range} is not a valid {ParameterRange.__name__}'
            if not param_range.is_valid(param_value):
                raise ValueError(
                    f'{get_default(param_value, "`None`")} is not a valid value for param '
                    f'`{param_name}`. {param_range.help_msg()}'
                )

    class TrieNode:
        def __init__(self, parent=None, value=None, children=None):
            self.parent = parent
            self.is_path_end = False
            self.value = value
            self.children: Dict[str, Any] = get_default(children, {})
            self.depth = self.get_depth(self)

        def __eq__(self, other):
            return self.is_path_end == other.is_path_end and \
                   self.children == other.children and \
                   self.parent == other.parent

        def __getitem__(self, key):
            return self.children[key]

        @classmethod
        def get_depth(cls, cur_node):
            """Calculates and returns depth of the current node. Root has depth of 0"""
            depth = 0
            while cur_node.parent is not None:
                depth += 1
                cur_node = cur_node.parent
            return depth

    @classmethod
    def create_trie(cls, string_list: List[str], splitter: str = '.', allow_end_at_branch: bool = True) -> TrieNode:
        """
        Creates a trie from a list of strings.
        Each node in the trie is a dict with further subdicts. Leafs are identified as dicts with '__end__' in them.
        Ref: https://stackoverflow.com/a/11016430
        """
        LIST_OF_STRINGS.check_is_valid(string_list)
        trie = cls.TrieNode()
        for string in string_list:
            current_node = trie
            string_split: List[str] = string.split(splitter)
            for string_part_i, string_part in enumerate(string_split):
                current_node.children.setdefault(string_part, cls.TrieNode(current_node))
                current_node = current_node.children[string_part]
                if string_part_i != len(string_split) - 1 and not allow_end_at_branch:
                    if current_node.is_path_end is True:
                        raise ValueError(
                            f'Branch nodes cannot be values for this Trie; thus cannot create trie from {string_list}'
                        )
            current_node.is_path_end = True
        return trie


from typing import Dict, List, Any, Union, Tuple, Set, Type
from ast import literal_eval
import math, re, json, sys, inspect
import numpy as np
from types import FunctionType
from datetime import datetime
from hashlib import sha256

StructuredBlob = Union[List, Dict, List[Dict]]  ## used for type hints.


class StringUtils:
    EMPTY = ''
    SPACE = ' '
    DOUBLE_SPACE = SPACE * 2
    FOUR_SPACE = SPACE * 4
    TAB = '\t'
    NEWLINE = '\n'
    WINDOWS_NEWLINE = '\r'
    BACKSLASH = '\\'
    SLASH = '/'
    PIPE = '|'
    SINGLE_QUOTE = "'"
    DOUBLE_QUOTE = '"'
    COMMA = ','
    COMMA_SPACE = ', '
    COMMA_NEWLINE = ',\n'
    HYPHEN = '-'
    DOUBLE_HYPHEN = '--'
    DOT = '.'
    ASTERISK = '*'
    QUESTION_MARK = '?'
    CARET = '^'
    DOLLAR = '$'
    UNDERSCORE = '_'
    COLON = ':'
    SEMICOLON = ';'
    EQUALS = '='
    LEFT_PAREN = '('
    RIGHT_PAREN = ')'
    BACKTICK = '`'

    MATCH_ALL_REGEX_SINGLE_LINE = CARET + DOT + ASTERISK + DOLLAR
    MATCH_ALL_REGEX_MULTI_LINE = DOT + ASTERISK

    S3_PREFIX = 's3://'
    PORT_REGEX = ':(\d+)'
    DOCKER_REGEX = '\d+\.dkr\.ecr\..*.amazonaws\.com/.*'

    DEFAULT_CHUNK_NAME_PREFIX = 'part'

    FILES_TO_IGNORE = ['_SUCCESS', '.DS_Store']

    UTF_8 = 'utf-8'

    FILE_SIZE_UNITS = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")

    ALPHABET = tuple('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

    ## Taken from: https://github.com/django/django/blob/master/django/utils/baseconv.py#L101
    BASE2_ALPHABET = '01'
    BASE16_ALPHABET = '0123456789ABCDEF'
    BASE56_ALPHABET = '23456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnpqrstuvwxyz'
    BASE36_ALPHABET = '0123456789abcdefghijklmnopqrstuvwxyz'
    BASE62_ALPHABET = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
    BASE64_ALPHABET = BASE62_ALPHABET + '-_'

    class BaseConverter:
        decimal_digits = '0123456789'

        def __init__(self, digits, sign='-'):
            self.sign = sign
            self.digits = digits
            if sign in self.digits:
                raise ValueError('Sign character found in converter base digits.')

        def __repr__(self):
            return "<%s: base%s (%s)>" % (self.__class__.__name__, len(self.digits), self.digits)

        def encode(self, i):
            neg, value = self.convert(i, self.decimal_digits, self.digits, '-')
            if neg:
                return self.sign + value
            return value

        def decode(self, s):
            neg, value = self.convert(s, self.digits, self.decimal_digits, self.sign)
            if neg:
                value = '-' + value
            return int(value)

        def convert(self, number, from_digits, to_digits, sign):
            if str(number)[0] == sign:
                number = str(number)[1:]
                neg = 1
            else:
                neg = 0

            # make an integer out of the number
            x = 0
            for digit in str(number):
                x = x * len(from_digits) + from_digits.index(digit)

            # create the result in base 'len(to_digits)'
            if x == 0:
                res = to_digits[0]
            else:
                res = ''
                while x > 0:
                    digit = x % len(to_digits)
                    res = to_digits[digit] + res
                    x = int(x // len(to_digits))
            return neg, res

    BASE_CONVERTER_MAP = {
        2: BaseConverter(BASE2_ALPHABET),
        16: BaseConverter(BASE16_ALPHABET),
        36: BaseConverter(BASE36_ALPHABET),
        56: BaseConverter(BASE56_ALPHABET),
        62: BaseConverter(BASE62_ALPHABET),
        64: BaseConverter(BASE64_ALPHABET, sign='$'),
    }

    def __init__(self):
        raise TypeError(f'Cannot instantiate class "StringUtils"')

    @classmethod
    def assert_not_empty_and_strip(cls, string, error_message='') -> str:
        cls.assert_not_empty(string, error_message)
        return string.strip()

    @classmethod
    def strip_if_not_empty(cls, string) -> str:
        if cls.is_not_empty(string):
            return string.strip()
        return string

    @classmethod
    def is_not_empty(cls, string: str) -> bool:
        return isinstance(string, str) and len(string.strip()) > 0

    @classmethod
    def is_not_empty_bytes(cls, string: bytes) -> bool:
        return isinstance(string, bytes) and len(string.strip()) > 0

    @classmethod
    def is_not_empty_str_or_bytes(cls, string: Union[str, bytes]) -> bool:
        return cls.is_not_empty(string) or cls.is_not_empty_bytes(string)

    @classmethod
    def is_empty(cls, string) -> bool:
        return not cls.is_not_empty(string)

    @classmethod
    def is_empty_bytes(cls, string) -> bool:
        return not cls.is_not_empty_bytes(string)

    @classmethod
    def is_empty_str_or_bytes(cls, string) -> bool:
        return not cls.is_not_empty_str_or_bytes(string)

    @classmethod
    def assert_not_empty(cls, string, error_message=''):
        assert cls.is_not_empty(string), error_message

    @classmethod
    def assert_not_empty_bytes(cls, string, error_message=''):
        assert cls.is_not_empty_str_or_bytes(string), error_message

    @classmethod
    def assert_not_empty_str_or_bytes(cls, string, error_message=''):
        assert cls.is_not_empty_str_or_bytes(string), error_message

    @classmethod
    def is_int(cls, string: str) -> bool:
        """
        Checks if an input string is an integer.
        :param string: input string
        :raises: error when input is not a string
        :return: True for '123', '-123' but False for '123.0', '1.23', '-1.23' and '1e2'
        """
        cls.assert_not_empty(string)
        try:
            int(string)
            return True
        except Exception as e:
            return False

    @classmethod
    def is_float(cls, string: str) -> bool:
        """
        Checks if an input string is a floating-point value.
        :param string: input string
        :raises: error when input is not a string
        :return: True for '123', '1.23', '123.0', '-123', '-123.0', '1e2', '1.23e-5', 'NAN' & 'nan'; but False for 'abc'
        """
        cls.assert_not_empty(string)
        try:
            float(string)  ## Will return True for NaNs as well.
            return True
        except Exception as e:
            return False

    @classmethod
    def is_prefix(cls, prefix: str, strings: Union[List[str], Set[str]]) -> bool:
        cls.assert_not_empty(prefix)
        if isinstance(strings, str):
            strings = [strings]
        return True in {string.startswith(prefix) for string in strings}

    @classmethod
    def remove_prefix(cls, string: str, prefix: str) -> str:
        cls.assert_not_empty(string)
        cls.assert_not_empty(prefix)
        if string.startswith(prefix):
            string = string[len(prefix):]
        return string

    @classmethod
    def convert_str_to_type(cls, val: str, expected_type: Type) -> Any:
        assert isinstance(expected_type, type)
        if isinstance(val, expected_type):
            return val
        if expected_type == str:
            return str(val)
        if expected_type == bool and isinstance(val, str):
            val = val.lower().strip().capitalize()  ## literal_eval does not parse "false", only "False".
        out = literal_eval(StringUtils.assert_not_empty_and_strip(str(val)))
        if expected_type == float and isinstance(out, int):
            out = float(out)
        if expected_type == int and isinstance(out, float) and int(out) == out:
            out = int(out)
        if expected_type == tuple and isinstance(out, list):
            out = tuple(out)
        if expected_type == list and isinstance(out, tuple):
            out = list(out)
        if expected_type == bool and out in [0, 1]:
            out = bool(out)
        if type(out) != expected_type:
            raise ValueError(f'Input value {val} cannot be converted to {str(expected_type)}')
        return out

    @classmethod
    def convert_bytes_to_human_readable(cls, size_in_bytes: int, num_decimals=3) -> str:
        sizes: Dict[str, float] = cls.convert_size_from_bytes(size_in_bytes, unit=None, num_decimals=num_decimals)
        sorted_sizes: List[Tuple[str, float]] = [
            (k, v) for k, v in sorted(sizes.items(), key=lambda item: item[1])
        ]
        size_unit, size_val = None, None
        for size_unit, size_val in sorted_sizes:
            if size_val >= 1:
                break
        return f'{size_val} {size_unit}'

    @classmethod
    def convert_size_from_bytes(cls, size_in_bytes: int, unit: str = None, num_decimals=3) -> Union[Dict, float]:
        size_in_bytes = float(size_in_bytes)
        cur_size = size_in_bytes
        sizes = {}
        for size_name in cls.FILE_SIZE_UNITS:
            val = round(cur_size, num_decimals)
            i = 1
            while val == 0:
                val = round(cur_size, num_decimals + i)
                i += 1
            sizes[size_name] = val
            i = int(math.floor(math.log(cur_size, 1024)))
            cur_size = cur_size / 1024
        if unit is not None:
            assert isinstance(unit, str)
            unit = unit.upper()
            assert unit in cls.FILE_SIZE_UNITS
            return sizes[unit]
        return sizes

    @classmethod
    def convert_size_to_bytes(cls, size_in_human_readable: str) -> int:
        size_in_human_readable: str = cls.assert_not_empty_and_strip(size_in_human_readable).upper()
        size_selection_regex = f'''(\d+(?:\.\d+)?) *({cls.PIPE.join(cls.FILE_SIZE_UNITS)})'''  ## This uses a non-capturing group: https://stackoverflow.com/a/3512530/4900327
        matches = re.findall(size_selection_regex, size_in_human_readable)
        if len(matches) != 1 or len(matches[0]) != 2:
            raise ValueError(f'Cannot convert value "{size_in_human_readable}" to bytes.')
        val, unit = matches[0]
        val = float(val)
        for file_size_unit in cls.FILE_SIZE_UNITS:
            if unit == file_size_unit:
                return int(round(val))
            val = val * 1024
        raise ValueError(f'Cannot convert value "{size_in_human_readable}" to bytes.')

    @classmethod
    def convert_seconds_to_human_readable(cls, time_in_seconds: float, num_decimals=3) -> str:
        times: Dict[str, float] = cls.convert_time_from_seconds(time_in_seconds, unit=None, num_decimals=num_decimals)
        sorted_times: List[Tuple[str, float]] = [
            (k, v) for k, v in sorted(times.items(), key=lambda item: item[1])
        ]
        time_unit, time_val = None, None
        for time_unit, time_val in sorted_times:
            if time_val >= 1:
                break
        if num_decimals <= 0:
            time_val = int(time_val)
        return f'{time_val} {time_unit}'

    @classmethod
    def convert_time_from_seconds(cls, time_in_seconds: float, unit: str = None, num_decimals=3) -> Union[Dict, float]:
        TIME_UNITS = {
            "milliseconds": 1e-3,
            "seconds": 1,
            "mins": 60,
            "hours": 60 * 60,
            "days": 24 * 60 * 60
        }
        time_in_seconds = float(time_in_seconds)
        times: Dict[str, float] = {
            time_unit: round(time_in_seconds / TIME_UNITS[time_unit], num_decimals)
            for time_unit in TIME_UNITS
        }
        if unit is not None:
            assert isinstance(unit, str)
            unit = unit.lower()
            assert unit in TIME_UNITS
            return times[unit]
        return times

    @classmethod
    def minify_json(cls, blob: StructuredBlob) -> str:
        return json.dumps(blob, indent=None, separators=(cls.COMMA, cls.COLON))

    @classmethod
    def pad_int_with_leading_zeros(cls, i: int, max_i: int = None) -> str:
        assert isinstance(i, int) and i >= 0
        if max_i is None:
            return str(i)
        assert isinstance(max_i, int) and max_i >= i
        num_zeros = math.ceil(math.log10(max_i))  ## Ref: https://stackoverflow.com/a/51837162/4900327
        if max_i == 10 ** num_zeros:  ## If it is a power of 10
            num_zeros += 1
        return f'{i:0{num_zeros}}'

    @classmethod
    def generate_random_strings(
            cls, shape=(1,), min_num_chars=6, max_num_chars=6, spaces_prob: float = None, alphabet: Tuple = ALPHABET
    ) -> Union[str, np.ndarray]:
        assert isinstance(min_num_chars, int) and min_num_chars >= 0, f'min_num_chars must be a non-negative integer; ' \
                                                                      f'found: {str(min_num_chars)}'
        assert isinstance(max_num_chars, int) and max_num_chars >= 1, f'max_num_chars must be a positive integer; ' \
                                                                      f'found: {str(max_num_chars)}'
        assert min_num_chars <= max_num_chars, \
            f'Must have min_num_chars ({min_num_chars}) <= max_num_chars ({max_num_chars})'
        if spaces_prob is not None:
            assert isinstance(spaces_prob, float) and 0.0 < spaces_prob < 1.0, \
                f'spaces_prob must be a float in range (0.0, 1.0), and it cannot be 0.0 or 1.0. Found: {spaces_prob}'
            num_spaces_to_add: int = int(round(len(alphabet) * spaces_prob / (1 - spaces_prob), 0))
            alphabet = alphabet + num_spaces_to_add * (cls.SPACE,)

        ## Ref: https://stackoverflow.com/a/25965461/4900327
        random_alphabet_lists = np.random.choice(alphabet, shape + (max_num_chars,))
        random_strings = np.apply_along_axis(
            arr=random_alphabet_lists,
            func1d=lambda random_alphabet_list:
            ''.join(random_alphabet_list)[:np.random.randint(min_num_chars, max_num_chars + 1)],
            axis=len(shape),
        )
        if shape == (1,):
            return random_strings[0]
        return random_strings

    @classmethod
    def parse_datetime(cls, dt: Union[str, int, float, datetime]) -> datetime:
        if isinstance(dt, datetime):
            return dt
        elif type(dt) in [int, float]:
            return datetime.fromtimestamp(dt)
        elif isinstance(dt, str):
            return datetime.fromisoformat(dt)
        raise NotImplementedError(f'Cannot parse datetime from value {dt} with type {type(dt)}')

    @classmethod
    def convert_integer_to_base_n_str(cls, integer: int, base: int) -> str:
        assert isinstance(integer, int)
        assert isinstance(base, int) and base in cls.BASE_CONVERTER_MAP, \
            f'Param `base` must be an integer in {list(cls.BASE_CONVERTER_MAP.keys())}; found: {base}'
        return cls.BASE_CONVERTER_MAP[base].encode(integer)

    @classmethod
    def hash(cls, val: Union[str, int, float, List, Dict], max_len: int = 256, base: int = 62) -> str:
        """
        Constructs a hash of a JSON object or value.
        :param val: any valid JSON value (including str, int, float, list, and dict).
        :param max_len: the maximum length of the output hash (will truncate upto this length).
        :param base: the base of the output hash.
            Defaults to base56, which encodes the output in a ASCII-chars
        :return: SHA256 hash.
        """

        def hash_rec(val, base):
            if isinstance(val, list):
                return hash_rec(','.join([hash_rec(x, base=base) for x in val]), base=base)
            elif isinstance(val, dict):
                return hash_rec(
                    [
                        f'{hash_rec(k, base=base)}:{hash_rec(v, base=base)}'
                        for k, v in sorted(val.items(), key=lambda kv: kv[0])
                    ],
                    base=base
                )
            return cls.convert_integer_to_base_n_str(int(sha256(str(val).encode('utf8')).hexdigest(), 16), base=base)

        return hash_rec(val, base)[:max_len]

    
from typing import Dict, List, Union, Tuple, Optional
import io, os, errno, sys, glob, pathlib, traceback, math, copy, time, logging
# from eps_data_transformation_lib.util.StringUtils import StringUtils
# from eps_data_transformation_lib.util.CollectionsUtils import CollectionsUtils


class FileSystemUtil:
    def __init__(self):
        raise TypeError(f'Cannot instantiate class "{str(self.__class__)}"')

    @classmethod
    def exists(cls, path: str) -> bool:
        return pathlib.Path(path).exists()

    @classmethod
    def dir_exists(cls, path: str) -> bool:
        try:
            return pathlib.Path(path).is_dir()
        except OSError as e:
            if e.errno == errno.ENAMETOOLONG:
                return False
            raise e

    @classmethod
    def is_path_valid_dir(cls, local_path: str) -> bool:
        local_path = StringUtils.assert_not_empty_and_strip(local_path)
        return cls.dir_exists(local_path) or local_path.endswith(os.path.sep)

    @classmethod
    def file_exists(cls, path: str) -> bool:
        try:
            return pathlib.Path(path).is_file()
        except OSError as e:
            if e.errno == errno.ENAMETOOLONG:
                return False
            raise e

    @classmethod
    def check_file_exists(cls, path: str):
        if cls.file_exists(path) is False:
            raise FileNotFoundError(f'Could not find file at location "{path}"')

    @classmethod
    def check_dir_exists(cls, path: str):
        if cls.dir_exists(path) is False:
            raise FileNotFoundError(f'Could not find dir at location "{path}"')

    @classmethod
    def files_exist(cls, paths: List[str]) -> bool:
        for path in paths:
            if not cls.file_exists(path):
                logging.info(f'Path "{path}" does not exist')
                return False
        return True

    @classmethod
    def get_dir(cls, path: str) -> str:
        '''
        Returns the directory of the path. If the path is an existing dir, returns the input.
        :param path: input file or directory path.
        :return: The dir of the passed path. Always ends in '/'.
        '''
        out_dir = None
        path = StringUtils.assert_not_empty_and_strip(path)
        if cls.dir_exists(path):  ## Works for both /home/seldon and /home/seldon/
            out_dir = path if path.endswith(os.path.sep) else path + os.path.sep
        else:
            dir = os.path.dirname(path)
            out_dir = dir if dir.endswith(os.path.sep) else dir + os.path.sep
        return out_dir

    @classmethod
    def mkdir_if_does_not_exist(cls, path: str, err=False) -> bool:
        try:
            dir_path = cls.get_dir(path)
            if not cls.is_writable(dir_path):
                raise OSError(f'Insufficient permissions to create directory at path {path}')
            os.makedirs(dir_path, exist_ok=True)
            if not cls.dir_exists(dir_path):
                raise OSError(f'Could not create directory at path {path}')
            return True
        except Exception as e:
            traceback.print_exc()
            if err:
                raise e
        return False

    @classmethod
    def is_writable(cls, path: str) -> bool:
        '''
        `True` if the current user has sufficient permissions to create the passed path; `False` otherwise.
        Backs off to checking parent files until it hits the root (this handles cases where the path may not exist yet).
        Ref: modified from https://stackoverflow.com/a/34102855
        '''
        ## Parent directory of the passed path.
        dir = cls.get_dir(path)
        if cls.dir_exists(dir):
            return os.access(dir, os.W_OK)
        dir_parents = pathlib.Path(dir).parents
        for i in range(len(dir_parents)):
            if cls.dir_exists(dir_parents[i]):
                return os.access(dir_parents[i], os.W_OK)
        return False

    @classmethod
    def list_files_in_dir(cls, dir_path, file_glob=StringUtils.ASTERISK, ignored_files=None) -> List[str]:
        if ignored_files is None:
            ignored_files = []
        file_paths: List[str] = glob.glob(os.path.join(dir_path, file_glob))
        file_names_map: Dict[str, str] = {file_path: os.path.basename(file_path) for file_path in file_paths}
        file_names_map = CollectionsUtils.filter_dict_values(file_names_map, ignored_files, exclude=True)
        file_paths = sorted(list(file_names_map.keys()))
        return file_paths if len(file_paths) > 0 else []

    @classmethod
    def list_first_file_in_dir(cls, dir_path, file_glob=StringUtils.ASTERISK, ignored_files=None) -> Optional[str]:
        file_paths: List[str] = cls.list_files_in_dir(dir_path, file_glob=file_glob, ignored_files=ignored_files)
        return file_paths[0] if len(file_paths) > 0 else None

    @classmethod
    def list_only_file_in_dir(cls, dir_path, file_glob=StringUtils.ASTERISK, ignored_files=None) -> Optional[str]:
        if cls.file_exists(dir_path):
            return dir_path  ## Is actually a file
        file_paths: List[str] = cls.list_files_in_dir(dir_path, file_glob=file_glob, ignored_files=ignored_files)
        if len(file_paths) == 0:
            return None
        if len(file_paths) > 1:
            raise FileNotFoundError(f'Multiple matching files are present in the directory')
        return file_paths[0]

    @classmethod
    def get_file_size(cls, file_path: str, unit=None, num_decimals=3) -> Union[float, str]:
        file_path = StringUtils.assert_not_empty_and_strip(file_path)
        size_in_bytes = pathlib.Path(file_path).stat().st_size
        if unit is not None:
            return StringUtils.convert_size_from_bytes(size_in_bytes, unit=unit, num_decimals=num_decimals)
        return StringUtils.convert_bytes_to_human_readable(size_in_bytes, num_decimals=num_decimals)

    @classmethod
    def get_time_last_modified(cls, path: str, num_decimals=3):
        path = StringUtils.assert_not_empty_and_strip(path)
        assert cls.exists(path)
        return round(os.path.getmtime(path), num_decimals)

    @classmethod
    def get_last_modified_time(cls, path: str):
        assert cls.exists(path)
        return os.path.getmtime(path)

    @classmethod
    def get_seconds_since_last_modified(cls, path: str, num_decimals=3):
        return round(time.time() - cls.get_last_modified_time(path), num_decimals)

    @classmethod
    def get_file_str(cls, file_path) -> str:
        file_str: str = None
        try:
            with io.open(file_path, 'r') as inp:
                file_str = inp.read()
            StringUtils.assert_not_empty(file_str)
        except Exception as e:
            traceback.print_exc()
            return None
        return file_str

    @classmethod
    def get_file_bytes(cls, file_path) -> bytes:
        file_bytes: bytes = None
        try:
            with io.open(file_path, 'rb') as inp:
                file_bytes = inp.read()
            StringUtils.assert_not_empty_bytes(file_bytes)
        except Exception as e:
            traceback.print_exc()
            return None
        return file_bytes

    @classmethod
    def put_file_str(cls, file_path, file_str: str, err=False) -> bool:
        StringUtils.assert_not_empty(file_str)
        try:
            with io.open(file_path, 'w') as out:
                out.write(file_str)
        except Exception as e:
            traceback.print_exc()
            if err:
                raise e
            else:
                return False
        return True

    @classmethod
    def construct_path_of_file_in_dir(cls, local_path: str, name: str, file_ending: str):
        """
        If the path is a dir, uses the inputs to construct a file path. If path is a file, returns unchanged.
        :param local_path: path to dir (or file) on filesystem.
        :param name: name of the file.
        :param file_ending: a string of the file ending.
        :return: file path string.
        """
        if cls.is_path_valid_dir(local_path):
            file_name: str = StringUtils.assert_not_empty_and_strip(name) + \
                             StringUtils.assert_not_empty_and_strip(file_ending)
            return os.path.join(cls.get_dir(local_path), file_name)
        else:
            return local_path

    @classmethod
    def is_stream(cls, obj):
        return isinstance(obj, io.TextIOBase) and hasattr(obj, 'read')

In [6]:
np.array([len(elem['context_tokens']) for elem in elems[1:]]).max()

794

In [11]:
bioasq_full = elems[1:]

In [20]:
np.random.shuffle(bioasq_full)

In [24]:
len(bioasq_full)

1504

In [37]:
bioasq_train = bioasq_full[ :round(0.7*len(bioasq_full))]
bioasq_train.insert(0, {'header': {'dataset': 'BioASQ', 'split': 'train'}})
print(len(bioasq_train))
bioasq_train[:3]

1054


[{'header': {'dataset': 'BioASQ', 'split': 'train'}},
 {'context': "Drosophila melanogaster has a single Adar gene encoding a protein related to mammalian ADAR2 that edits transcripts encoding glutamate receptor subunits. We describe the structure of the Drosophila Adar locus and use ModENCODE information to supplement published data on Adar gene transcription, and splicing. We discuss the roles of ADAR in Drosophila in terms of the two main types of RNA molecules edited and roles of ADARs as RNA-binding proteins. Site-specific RNA editing events in transcripts encoding ion channel subunits were initially found serendipitously and subsequent directed searches for editing sites and transcriptome sequencing have now led to 972 edited sites being identified in 597 transcripts. Four percent of D. melanogaster transcripts are site-specifically edited and these encode a wide range of largely membrane-associated proteins expressed particularly in CNS. Electrophysiological studies on the effec

In [44]:
with gzip.open('datasets/bioasq_train.jsonl.gz', 'wb') as f:
    for ex in bioasq_train:
        ## Ref: https://stackoverflow.com/a/39451012
        f.write((json.dumps(ex)+'\n').encode('utf-8'))
        
with gzip.open('datasets/bioasq_dev.jsonl.gz', 'wb') as f:
    for ex in bioasq_dev:
        ## Ref: https://stackoverflow.com/a/39451012
        f.write((json.dumps(ex)+'\n').encode('utf-8'))

In [38]:
bioasq_dev = bioasq_full[round(0.7*len(bioasq_full)): ]
bioasq_dev.insert(0, {'header': {'dataset': 'BioASQ', 'split': 'dev'}})
print(len(bioasq_dev))
bioasq_dev[:3]

452


[{'header': {'dataset': 'BioASQ', 'split': 'dev'}},
 {'context': 'Dasatinib (BMS-354825) is a Src/ABL tyrosine kinase inhibitor currently approved for the treatment of chronic myeloid leukemia. Dasatinib has increased potency against ABL compared to the current therapy imatinib, and is effective in many cases where disease is resistant to imatinib. Dasatinib also inhibits many Src-family tyrosine kinases. We have demonstrated in this study that dasatinib is able to block the function of normal human T-lymphocytes in vitro at clinically relevant concentrations. T-cell functions including proliferation, activation and cytokine production were all uniformly inhibited in the presence of dasatinib. We also demonstrated inhibition of TCR signalling through Src-family kinase LCK, and predicted that inhibition of LCK and other kinases involved in T-cell signalling by dasatinib is responsible for the suppression of T-cell function. These findings raise the concern about potential T-cell inhibit

In [29]:
max_context_length = 5000
passage_tokens_idxs[]

['friend', 2223]

In [225]:
import numpy as np
for elem_i in np.random.permutation(np.arange(0, len(elems))[1:] ):
    elem = elems[elem_i]
    context = elem['context']
#     print(context)
    passage = [x[0] for x in elem['context_tokens']]
    print(passage)
    for qa in elem['qas']:
        print(qa)
        print(passage[qa['detected_answers'][0]['token_spans'][0][0]: 1 + qa['detected_answers'][0]['token_spans'][0][1]])
        break
    break

['(', 'OPRAH.com', ')', '--', 'He', 'can', 'saw', 'himself', 'in', 'half', ',', 'sing', 'a', 'selection', 'of', 'Broadway', 'showtunes', 'and', 'swing', 'on', 'a', 'flying', 'trapeze', '.', 'Neil', 'Patrick', 'Harris', 'says', 'he', "'ll", 'try', 'to', 'make', 'viewers', 'feel', 'like', 'they', "'re", 'in', 'good', 'hands', 'with', 'him', 'as', 'Emmy', 'host', '.', 'When', 'Neil', 'Patrick', 'Harris', ',', 'one', 'of', 'the', 'stars', 'of', 'the', 'hit', 'CBS', 'sitcom', '"', 'How', 'I', 'Met', 'Your', 'Mother', ',', '"', 'is', "n't", 'dabbling', 'in', 'the', 'extraordinary', ',', 'well', ',', 'he', "'s", 'probably', 'hosting', 'an', 'awards', 'show', '.', 'In', 'the', 'late', "'", '80s', ',', 'Neil', '--', 'known', 'as', 'NPH', 'to', 'his', 'fans', '--', 'landed', 'the', 'starring', 'role', 'on', '"', 'Doogie', 'Howser', ',', 'M.D.', '"', 'After', 'years', 'of', 'child', 'stardom', 'and', 'teen', 'heartthrob', 'status', ',', 'Neil', 'left', 'the', 'small', 'screen', 'for', 'the', 'sta

In [93]:
from typing import *
max_context_length = 10
passage_tokens_idxs: List[Tuple[str, int]] = elem['context_tokens']
passage_tokens: List[str] = [
    token.lower()
    for (token, offset) in passage_tokens_idxs
][:max_context_length]
print(passage_tokens_idxs[:max_context_length])
passage: str = elem['context']
final_passage_token_idx: int = min(max_context_length, len(passage_tokens_idxs)) - 1
final_passage_token__start_idx: int = passage_tokens_idxs[final_passage_token_idx][1]
final_passage_token_len = len(passage_tokens_idxs[final_passage_token_idx][0])
final_passage_idx = final_passage_token__start_idx + final_passage_token_len
passage: str = passage[:final_passage_idx]
print(f"<START>{passage}<END>")

[['Reuters', 0], ['is', 8], ['a', 11], ['global', 13], ['information', 20], ['company', 32], ['providing', 40], ['material', 50], ['tailored', 59], ['for', 68]]
<START>Reuters is a global information company providing material tailored for<END>


In [138]:
def get_ngrams(tokens: Union[str, List[str]], n_gram: int) -> Counter:
    assert isinstance(n_gram, int) and n_gram >= 1
    ngrams_counts = Counter()
    if n_gram == 1:
        for token in tokens:
            ngrams_counts[token] += 1
        return ngrams_counts

    for i in range(len(tokens) - n_gram + 1):
        n_gram_slice = tuple(tokens[i: i + n_gram])
        ngrams_counts[n_gram_slice] += 1
    return ngrams_counts
print(get_ngrams(passage_tokens, 2))
len(get_ngrams(passage_tokens, 2))

Counter({'--': 2, 'his': 2, 'the': 2, 'to': 2, '(CNN)': 1, 'A': 1, 'phone': 1, 'hacking': 1, 'scandal': 1, 'may': 1, 'have': 1, 'cost': 1, 'Rupert': 1, 'Murdoch': 1, 'biggest-selling': 1, 'newspaper': 1, 'in': 1, '2011': 1, ',': 1, 'but': 1, 'billionaire': 1, 'media': 1, 'mogul': 1, 'managed': 1, 'end': 1, 'year': 1, 'with': 1, 'a': 1, 'modest': 1, 'addition': 1, 'empire': 1, 'an': 1, 'account': 1, 'on': 1, 'Twitter': 1, '.': 1})


36

In [51]:
from typing import *
max_question_length = 5
question_tokens_idxs: List[Tuple[str, int]] = qa['question_tokens']
question_tokens: List[str] = [
    token.lower()
    for (token, offset) in question_tokens_idxs
][:max_question_length]
print(question_tokens_idxs[:max_question_length])
question: str = qa['question']
final_question_token_idx: int = min(max_question_length, len(question_tokens_idxs)) - 1
final_question_token_start_idx: int = question_tokens_idxs[final_question_token_idx][1]
final_question_token_len = len(question_tokens_idxs[final_question_token_idx][0])
final_question_idx = final_question_token_start_idx + final_question_token_len
question: str = question[:final_question_idx]
print(f"<START>{question}<END>")

[['Which', 0], ['President', 6], ["'s", 15], ['death', 18], ['was', 24]]
<START>Which President's death was<END>


In [10]:
# model_name = 'google/pegasus-large'
# model_name = 'google/pegasus-xsum'
# model_name = 'tuner007/pegasus_paraphrase'
model_name = 'google/pegasus-cnn_dailymail'
# model_name = 'google/pegasus-multi_news'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
if 'model' in globals():
    del model
    import gc; gc.collect()
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

In [11]:
def get_response(input_text, num_return_sequences, num_beams, max_length):
    if isinstance(input_text, str):
        input_text = [input_text]
    batch = tokenizer(input_text, truncation=True,padding='longest', max_length=max_length, return_tensors="pt").to(device)
    paraphrased_token_ids = model.generate(**batch,max_length=max_length, num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
#     print(paraphrased_token_ids)
    paraphrases = tokenizer.batch_decode(paraphrased_token_ids, skip_special_tokens=True)
    paraphrased_tokens: List[str] = [
        [## The character ▁ (NOT an underscore) is used by sentencepiece to represent a space.
        ## Ref: https://huggingface.co/transformers/tokenizer_summary.html#sentencepiece
            token#.replace('▁', '')
            for token in tokenizer.convert_ids_to_tokens(paraphrased_token_ids_list, skip_special_tokens=True)
        ]
#         tokenizer.decode(paraphrased_token_ids_list, skip_special_tokens=True, clean_up_tokenization_spaces=False, spaces_between_special_tokens=True).split(' ')
        for paraphrased_token_ids_list in paraphrased_token_ids
    ]
    del batch, paraphrased_token_ids
    print('Paraphrased tokens:')
    for paraphrased_token_list in paraphrased_tokens:
        print(paraphrased_token_list)
    return paraphrases, paraphrased_tokens

In [191]:
# tokenizer.decode(
#     ['▁Around', '▁midnight', '▁at', '▁London', "'", 's', '▁Leicester', '▁Square', ',', '▁as', '▁news', '▁of', '▁Jackson', "'", 's', '▁death', '▁spread', ',', '▁Luis', '▁Carlos', '▁A', 'me', 'ida', '▁and', '▁his', '▁friends', '▁were', '▁surrounding']
# )
# tokenizer.decode(
#     [0, 14915, 13635, 16923,  4195,     1],
#     skip_special_tokens=False,
#     clean_up_tokenization_spaces=False,
#     spaces_between_special_tokens=False,
# )

In [12]:
paraphrase = get_response(
#     """In Adelaide, Australia, Christos Winter of the MJ Fan Club had organized a petition to bring Jackson to tour there. "It didn't matter if you were 60, 40 or 20 like I am. Michael Jackson's music just spoke to everyone ... It was always uplifting and happy music," Winter told CNN.""",
#     """In Glastonbury, southern England, where one of the world's largest music festivals was to kick off Friday morning, initial rumors and then confirmation of Jackson's death added to confusion and then shock among festival goers.""",
    """Another iReporter, Peter Maiyoh, a Kenyan student studying in the U.S. city of Kansas, Missouri, called Jackson "the voice of change," saying "he was there before Tiger Woods, before Michael Jordan, even before Barack Obama ... I hope people remember him for the work he did.""",
#     """Around midnight at London's Leicester Square, as news of Jackson's death spread, Luis Carlos Ameida and his friends were surrounding a car listening to the star's music. Ameida said he'd gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13 in London.""", 
    10, 10, 120
)[0][0]

Paraphrased tokens:
['▁Peter', '▁Mai', 'y', 'oh', '▁is', '▁a', '▁Kenyan', '▁student', '▁studying', '▁in', '▁the', '▁U', '.', 'S', '.', '▁city', '▁of', '▁Kansas', ',', '▁Missouri', '▁', '.', '<n>', 'He', '▁says', '▁Jackson', '▁"', 'was', '▁there', '▁before', '▁Tiger', '▁Woods', ',', '▁before', '▁Michael', '▁Jordan', ',', '▁even', '▁before', '▁Barack', '▁Obama', '"', '<n>', 'Mai', 'y', 'oh', ':', '▁"', 'I', '▁hope', '▁people', '▁remember', '▁him', '▁for', '▁the', '▁work', '▁he', '▁did', '"']
['▁Peter', '▁Mai', 'y', 'oh', '▁is', '▁a', '▁Kenyan', '▁student', '▁studying', '▁in', '▁the', '▁U', '.', 'S', '.', '▁city', '▁of', '▁Kansas', ',', '▁Missouri', '▁', '.', '<n>', 'He', '▁says', '▁Jackson', '▁"', 'was', '▁there', '▁before', '▁Tiger', '▁Woods', ',', '▁before', '▁Michael', '▁Jordan', ',', '▁even', '▁before', '▁Barack', '▁Obama', '"', '<n>', 'Mai', 'y', 'oh', ':', '▁I', '▁hope', '▁people', '▁remember', '▁him', '▁for', '▁the', '▁work', '▁he', '▁did', '.']
['▁Peter', '▁Mai', 'y', 'oh', ',', 

In [212]:
paraphrase

'Peter Maiyoh is a Kenyan student studying in the U.S. city of Kansas, Missouri.<n>He says Jackson "was there before Tiger Woods, before Michael Jordan, even before Barack Obama"<n>Maiyoh: "I hope people remember him for the work he did"'

In [88]:
tokenizer.convert_tokens_to_ids(passage) 

[741,
 40155,
 158,
 105,
 55229,
 57810,
 11388,
 4378,
 44041,
 20388,
 2130,
 544,
 65636,
 108,
 34045,
 108,
 4581,
 1313,
 304,
 105,
 105,
 105,
 4468,
 304,
 26419,
 1313,
 2449,
 18107,
 14866,
 6479,
 108,
 12152,
 22023,
 11339,
 105,
 41732,
 20388,
 107,
 39892,
 661,
 20388,
 30086,
 36313,
 105,
 3030,
 10072,
 105,
 1313,
 105,
 74283,
 8109,
 105,
 544,
 44041,
 1313,
 23958,
 2130,
 13062,
 4581,
 14866,
 6479,
 108,
 60108,
 41732,
 107,
 105,
 108,
 4442,
 108,
 8161,
 10140,
 30086,
 3030,
 105,
 108,
 105,
 92425,
 526,
 105,
 108,
 60108,
 41732,
 107,
 5277,
 105,
 1659,
 57824,
 1897,
 20604,
 37787,
 107,
 12152,
 105,
 105,
 661,
 13280,
 108,
 8996,
 19748,
 8109,
 105,
 544,
 105,
 44041,
 1313,
 10072,
 105,
 33374,
 13062,
 4581,
 526,
 23612,
 19973,
 105,
 10234,
 30665,
 544,
 2907,
 107,
 12800,
 20388,
 108,
 304,
 9534,
 18301,
 497,
 544,
 105,
 13586,
 105,
 40155,
 3713,
 304,
 105,
 10234,
 13586,
 34762,
 22258,
 386,
 544,
 4581,
 107,
 159,
 

In [90]:
__input_text = context
__batch = tokenizer(__input_text,truncation=True, padding='longest',max_length=len(__input_text), return_tensors="pt").to(device)
__translated = model.generate(**__batch, max_length=len(__input_text), num_beams=10, num_return_sequences=10, temperature=1.5)

In [ ]:
model.generate(**__batch, max_length=len(__input_text), num_beams=10, num_return_sequences=10, temperature=1.5)

In [91]:
__translated

tensor([[    0,  6053,   151, 24643,  2058,   541,   154,  3573,   134,  6786,
           412,  3422,   131,   116,   238,   110,   107,   106, 17321,   151,
           412,  3422,  3131,   122,   668,  7751,   113, 30459,  5939,   108,
          1564,   416,   110,   107,   106, 13157,   151,   624,   307,   513,
           374,   134,   412,  3422,   131,   116,  7894,   108,  3698,   108,
           238,   110,   107,   106, 11712, 21298,   256,   133,   174,   186,
           118,   198,  6479,   116,   108,   175,   146,   590,   132,   231,
           745, 53898,   649,   110,   107,     1,     0,     0],
        [    0,  6053,   151, 24643,  2058,   541,   154,  3573,   134,  6786,
           412,  3422,   131,   116,   238,   110,   107,   106, 17321,   151,
           412,  3422,  3131,   122,   668,  7751,   113, 30459,  5939,   108,
          1564,   416,   110,   107,   106, 11712, 21298,   113,  1029,   652,
           374,   134,   412,  3422,   131,   116,  7894,   108, 

In [61]:
# tokenizer.convert_tokens_to_string(
[
    token.replace('▁', '')
    for token in 
    tokenizer.convert_ids_to_tokens(__translated[0], skip_special_tokens=True)
]
# )

['Glastonbury',
 ',',
 'one',
 'of',
 'the',
 'world',
 "'",
 's',
 'largest',
 'music',
 'festivals',
 ',',
 'was',
 'to',
 'kick',
 'off',
 'Friday',
 'morning',
 '',
 '.',
 '<n>',
 'In',
 'southern',
 'England',
 ',',
 'where',
 'one',
 'of',
 'the',
 'world',
 "'",
 's',
 'largest',
 'music',
 'festivals',
 'was',
 'to',
 'kick',
 'off',
 'Friday',
 'morning',
 ',',
 'initial',
 'rumors',
 'and',
 'then',
 'confirmation',
 'of',
 'Jackson',
 "'",
 's',
 'death',
 'added',
 'to',
 'confusion',
 'and',
 'shock',
 'among',
 'festival',
 'goers',
 '.']

In [56]:
'▁' == '_'

False

In [8]:
# import time
# start = time.time()
# get_response([
#     """In Glastonbury, southern England, where one of the world's largest music festivals was to kick off Friday morning, initial rumors and then confirmation of Jackson's death added to confusion and then shock among festival goers.""",
# #     """On a street in New Delhi, India, 31-year-old Sachina Verma said on Friday, "Any of the baby boomer generation or, you know, people from my age or our time, I mean they have grown up on his music. Literally, people have been inspired by his dance movements, by his music.""",
# #     """In Adelaide, Australia, Christos Winter of the MJ Fan Club had organized a petition to bring Jackson to tour there. "It didn't matter if you were 60, 40 or 20 like I am. Michael Jackson's music just spoke to everyone ... It was always uplifting and happy music," Winter told CNN.""",
# #     """Another iReporter, Peter Maiyoh, a Kenyan student studying in the U.S. city of Kansas, Missouri, called Jackson "the voice of change," saying "he was there before Tiger Woods, before Michael Jordan, even before Barack Obama ... I hope people remember him for the work he did.""",
# #     """Around midnight at London's Leicester Square, as news of Jackson's death spread, Luis Carlos Ameida and his friends were surrounding a car listening to the star's music. Ameida said he'd gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13 in London.""",
# ], num_return_sequences=10, num_beams=10, max_length=int(1.5*51))
# end = time.time()
# print(f'{end-start:.3f}')

In [217]:
word_tokenize('lol what the fuck; I thought? Huh, I guessed I never...whatever, man.')

['lol',
 'what',
 'the',
 'fuck',
 ';',
 'I',
 'thought',
 '?',
 'Huh',
 ',',
 'I',
 'guessed',
 'I',
 'never',
 '...',
 'whatever',
 ',',
 'man',
 '.']

In [218]:
elems[0]

{'header': {'dataset': 'NewsQA', 'split': 'dev'}}

In [106]:
import re
stopwords = {'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
from utils import lcs
def get_paraphrase(context, num_beams=75, len_multiplier=1.5):
#     print(num_beams)
    context = re.sub('(\n)+', '\n', context)
    context = re.sub('( )+', ' ', context)
    responses = get_response(context, num_return_sequences=num_beams, num_beams=num_beams, max_length=int(len_multiplier*len(context.split(' '))))
    responses = [response.replace('  ', ' ') for response in responses]
#     print(len(responses))
    paraphrase_scores = []
    for response in responses:
        response = response.replace('<n>', ' ')
#         print(response)
#         print()
        overlap_score = lcs(context.split(' '), response.split(' '))[0]
        unique_score = len(set(response.split(' ')))  ## Can change to unique bigrams score
        num_unseen_unigrams = len(set(context.split(' ')).union(set(response.split(' '))) - set(context.split(' ')) - stopwords)
        paraphrase_score = round(unique_score / (overlap_score + num_unseen_unigrams), 3)
#         print('Overlap between orig and paraphrase', overlap_score)
#         print('No. unique tokens in paraphrase: ', unique_score)
#         print('paraphrase score: ', paraphrase_score)
        paraphrase_scores.append((paraphrase_score, response))
    paraphrase_scores = list(set(paraphrase_scores))
#     print('='*50)
    return sorted(paraphrase_scores, key=lambda x: x[0], reverse=True)
def print_paraphrase(context, num_beams=75, len_multiplier=1.5,):    
    import time
    start = time.time()
    for x in get_paraphrase(context, num_beams=num_beams, len_multiplier=len_multiplier):
        print(x)
    end = time.time()
    print(f'{end-start:.3f}')

In [13]:
1+1

2

In [108]:
import re, time
NORMALIZED_SENT_LEN = 20  ## words
elem = elems[11]
passage_tokens = [x[0] for x in elem['context_tokens']]
context = elem['context']
context = re.sub('(\n)+', '\n', context)
context = re.sub('( )+', ' ', context)
context = context.split('\n')

context_sents_split = []
for cx_para in context:
    if len(cx_para.split(' ')) < NORMALIZED_SENT_LEN:
        context_sents_split.append(cx_para)
    else:
        cx_para_concats = []
        for line in cx_para.split('. '):
            if not line.endswith('.'):
                line += '.'
            if len(cx_para_concats) == 0:
                cx_para_concats.append(line)
            else:
                if len(cx_para_concats[-1].split(' ')) < NORMALIZED_SENT_LEN:
                    cx_para_concats[-1] += ' ' + line
                else:
                    cx_para_concats.append(line)
        context_sents_split.extend(cx_para_concats)
context = context_sents_split
    
context_sents_concat = []
for cx_para in context:
    if len(context_sents_concat) == 0:
        context_sents_concat.append(cx_para)
    else:
        if len(context_sents_concat[-1].split(' ')) < NORMALIZED_SENT_LEN:
            context_sents_concat[-1] += ' ' + cx_para
        else:
            context_sents_concat.append(cx_para)
context = context_sents_concat

start = time.time()
for paragraph in context:
    print('='*50, end='\n\n')
    print('Original:\n')
    print(paragraph)
    print('\nParaphrased:\n')
    for x in get_paraphrase(
        paragraph,
        num_beams=75,
        len_multiplier=1.5,
    )[0:3]:
        print(x)
end = time.time()
print(f'Time taken: {end-start:.3f}')

print()
print('='*80)
print('='*80)
print('='*80)
print()

questions = elem['qas']
for question in questions:
    print('\n' + '='*50 + f'\nquestion: {question["question"]}')
    detected_answers = question['detected_answers']

    print(f'\ndetected_answers:')
    for detected_ans_i, detected_ans in enumerate(detected_answers):
        print(f'{detected_ans_i+1}) {detected_ans["text"]}')
        span_idxs = (detected_ans['token_spans'][0][0], detected_ans['token_spans'][0][1]+1)
        span = passage_tokens[span_idxs[0]: span_idxs[1]]
        print(f'\nAnswer span: {span}')



Original:

(CNN) -- A phone hacking scandal may have cost Rupert Murdoch his biggest-selling newspaper in 2011, but the billionaire media mogul managed to end the year with a modest addition to his empire -- an account on Twitter.

Paraphrased:

(1.643, 'Rupert Murdoch opened a Twitter account in 2011. The account was set up in the wake of the phone hacking scandal that rocked his media empire.')
(1.632, "Rupert Murdoch opened a Twitter account in 2011. The account was set up in the wake of the phone hacking scandal that rocked his media empire. Murdoch is one of the world's richest and most influential media moguls.")
(1.632, "Rupert Murdoch launched a Twitter account in 2011. The account was set up in the wake of the phone hacking scandal that rocked his media empire. Murdoch is one of the world's richest and most influential media moguls.")

Original:

Within 48 hours of debuting with tweets about family, work and politics, Murdoch had pulled in more than 45,000 followers and stirr

In [141]:
n_gram = 1
stopwords_and_punctuation = {',', ';', ':', '.', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
passage_tokens = '(CNN) -- A phone hacking scandal may have cost Rupert Murdoch his biggest-selling newspaper in 2011 , but the billionaire media mogul managed to end the year with a modest addition to his empire -- an account on Twitter .'.split(' ')
orig_ngrams = get_ngrams(passage_tokens, n_gram)
paraphrased_tokens = 'Rupert Murdoch opened a Twitter account in 2011. The account was set up in the wake of the phone hacking scandal that rocked his media empire.'.split(' ')
paraphrased_ngrams = get_ngrams(paraphrased_tokens, n_gram)
print(f'Original    #unique ngrams: {len(orig_ngrams)}')
print(f'Paraphrased #unique ngrams: {len(paraphrased_ngrams)}')
hallucinated_unigrams = set(paraphrased_ngrams.keys()) - set(orig_ngrams.keys()) - stopwords_and_punctuation
print(f'Hallcinated unigrams: {len(hallucinated_unigrams)}: {hallucinated_unigrams}')

Original    #unique ngrams: 39
Paraphrased #unique ngrams: 25
Hallcinated unigrams: 22: {('account', 'in'), ('rocked', 'his'), ('media', 'empire.'), ('set', 'up'), ('in', 'the'), ('his', 'media'), ('scandal', 'that'), ('The', 'account'), ('Twitter', 'account'), ('2011.', 'The'), ('a', 'Twitter'), ('the', 'phone'), ('the', 'wake'), ('account', 'was'), ('of', 'the'), ('was', 'set'), ('that', 'rocked'), ('wake', 'of'), ('opened', 'a'), ('Murdoch', 'opened'), ('up', 'in'), ('in', '2011.')}


In [ ]:
# print_paraphrase(
#     """In Glastonbury, southern England, where one of the world's largest music festivals was to kick off Friday morning, initial rumors and then confirmation of Jackson's death added to confusion and then shock among festival goers.""",
#     num_beams=50,
#     len_multiplier=1.5,
# )

In [10]:
input_string = ["Pegasus is <mask_2> . <mask_1> it <mask_2> the model ."]

In [23]:
decoder_input_string = ["<s> It is pure white . "]

In [24]:
labels_string = ["It is pure white . </s>"]

In [25]:
input_ids = tokenizer(input_string, add_special_tokens=False, return_tensors="pt").input_ids.to(device)
print(input_ids)
decoder_input_ids = tokenizer(decoder_input_string, add_special_tokens=False, return_tensors="pt", bos_token='<s>').input_ids.to(device)
print(decoder_input_ids)
labels = tokenizer(labels_string, add_special_tokens=False, return_tensors="pt").input_ids.to(device)
print(labels)

Keyword arguments {'bos_token': '<s>'} not recognized.


tensor([[51881,   117,     3,   110,   107,     2,   126,     3,   109,   861,
           110,   107]], device='cuda:0')
tensor([[ 110,  105,  116, 2314,  168,  117, 3763,  695,  110,  107]],
       device='cuda:0')
tensor([[ 168,  117, 3763,  695,  110,  107,    1]], device='cuda:0')


In [10]:
from utils import lcs
lcs('xyzhat sourdough', 'thatso urmantra')

(7, 'hatsour')

In [59]:
! pip install nltk spacy

     |████████████████████████████████| 1.5 MB 4.3 MB/s eta 0:00:01


In [7]:
tokenizer.get_vocab()

NameError: name 'tokenizer' is not defined

In [71]:
for x in get_paraphrase("""In Glastonbury, southern England, where one of the world's largest music festivals was to kick off Friday morning, initial rumors and then confirmation of Jackson's death added to confusion and then shock among festival goers."""):
    print(x)

(1.025, "In Glastonbury, southern England, one of the world's largest music festivals was to kick off Friday morning. Initial rumors and then confirmation of Jackson's death added to confusion and then shock among festival goers. The festival is one of the biggest in the world, drawing people from all over the world.")
(1.024390243902439, "In Glastonbury, southern England, one of the world's largest music festivals was to kick off Friday morning. Initial rumors and then confirmation of Jackson's death added to confusion and then shock among festival goers. The three-day festival is one of the biggest in the world, drawing people from all over the world.")
(1.0238095238095237, "In Glastonbury, southern England, one of the world's largest music festivals was to kick off Friday morning. Initial rumors and then confirmation of Jackson's death added to confusion and then shock among festival goers. The three-day festival is one of the biggest in the world, drawing more than 200,000 people e

In [73]:
print_paraphrase("""Around midnight at London's Leicester Square, as news of Jackson's death spread, Luis Carlos Ameida and his friends were surrounding a car listening to the star's music. Ameida said he'd gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13 in London.""")

(1.56, 'Luis Carlos Ameida was in London\'s Leicester Square at the time of Jackson\'s death. Ameida said he\'d gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13. Ameida said he and his friends were surrounding a car listening to Jackson\'s music.')
(1.5384615384615385, 'Luis Carlos Ameida was in London\'s Leicester Square when news of Jackson\'s death spread. Ameida said he\'d gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13. Ameida said he and his friends were surrounding a car listening to Jackson\'s music.')
(1.5185185185185186, 'Luis Carlos Ameida was at London\'s Leicester Square when news of Jackson\'s death spread. Ameida said he\'d gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13. Ameida said he and his friends were surrounding a car listening to the star\'s music.')
(1.5, 'Luis Carlos Ameida and friends were in London\'s Leicester Square when news of Jackson\'s death spread. Ameida sai

In [98]:
print_paraphrase("""On a street in New Delhi, India, 31-year-old Sachina Verma said on Friday, "Any of the baby boomer generation or, you know, people from my age or our time, I mean they have grown up on his music. Literally, people have been inspired by his dance movements, by his music.""")

75
(1.0357142857142858, '"Any of the baby boomer generation or, you know, people from my age or our time," Sachina Verma says. People have been inspired by his dance movements, by his music, she says.')
(1.0344827586206897, '"Any of the baby boomer generation or, you know, people from my age or our time," Sachina Verma says. "People have been inspired by his dance movements, by his music," she adds.')
(1.0344827586206897, '"Any of the baby boomer generation or, you know, people from my age or our time," Sachina Verma says. People have been inspired by his dance movements, by his music, she adds.')
(1.0, '"Any of the baby boomer generation or, you know, people from my age or our time," says Sachina Verma. People have been inspired by his dance movements, by his music, she says.')
(1.0, '"Any of the baby boomer generation or, you know, people from my age or our time," Sachina Verma says. " Literally, people have been inspired by his dance movements, by his music," she adds.')
(1.0, '"Any

In [157]:
import re, random
context = elem['context']

context = context.split('\n')
random.shuffle(context)
context = context[0]
context = """Rincewind had always been happy to think of himself as a racist.  The One Hundred Metres, the Mile, the Marathon – he'd run them all."""
context = """In Adelaide, Australia, Christos Winter of the MJ Fan Club had organized a petition to bring Jackson to tour there. "It didn't matter if you were 60, 40 or 20 like I am. Michael Jackson's music just spoke to everyone ... It was always uplifting and happy music," Winter told CNN."""
context = """In Glastonbury, southern England, where one of the world's largest music festivals was to kick off Friday morning, initial rumors and then confirmation of Jackson's death added to confusion and then shock among festival goers."""
context = """Another iReporter, Peter Maiyoh, a Kenyan student studying in the U.S. city of Kansas, Missouri, called Jackson "the voice of change," saying "he was there before Tiger Woods, before Michael Jordan, even before Barack Obama ... I hope people remember him for the work he did."""
context = """Expressing sadness and shock over Jackson's death, Ng recalled being inspired by an interview he once gave to Oprah Winfrey. "He said if you have power, try to give it back and help the others, and I will try to do that," she said."""
context = """Around midnight at London's Leicester Square, as news of Jackson's death spread, Luis Carlos Ameida and his friends were surrounding a car listening to the star's music. Ameida said he'd gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13 in London."""
# context = elem['context']
# context = '.'.join(context.split('.')[0:3]) + '.'

context = re.sub('(\n)+', '\n', context)
context = re.sub('( )+', ' ', context)

responses = get_response(context, num_return_sequences=100, num_beams=100, max_length=int(1.5*len(context.split(' '))))
print('\n')
print(context)
print('\n')
responses = [response.replace('  ', ' ') for response in responses]
print(responses)
print(len(responses))
paraphrase_scores = []
for response in responses:
    response = response.replace('<n>', ' ')
    print('='*50)
    print(response)
    overlap_score = lcs(context.split(' '), response.split(' '))[0]
    unique_score = len(set(response.split(' ')))  ## Can change to unique bigrams score
    paraphrase_score = unique_score / overlap_score
    print('Overlap between orig and paraphrase', overlap_score)
    print('No. unique tokens in paraphrase: ', unique_score)
    print('paraphrase score: ', paraphrase_score)
    paraphrase_scores.append([paraphrase_score, response])

# sorted(paraphrase_scores, key=lambda x: x[0], reverse=True)



Around midnight at London's Leicester Square, as news of Jackson's death spread, Luis Carlos Ameida and his friends were surrounding a car listening to the star's music. Ameida said he'd gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13 in London.


['Luis Carlos Ameida got tickets to see Jackson at his "This Is It" concerts.<n>Ameida said he\'d gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13 in London.<n>Ameida said he\'d gotten tickets to see Jackson at his "This', 'Ameida said he\'d gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13 in London.<n>Ameida said he\'d gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13 in London.', 'Luis Carlos Ameida got tickets to see Jackson at his "This Is It" concerts.<n>Ameida said he\'d gotten tickets to see Jackson at his "This Is It" concerts beginning on July 13.<n>Ameida said he\'d gotten tickets to see Jackson at his "This 

In [83]:
print_paraphrase("""Obama also named Lisa Jackson, former head of New Jersey's environmental agency, to serve as his Environmental Protection Agency administrator, and Nancy Sutley, the Los Angeles deputy mayor for energy and environment, to lead the White House Council on Environmental Quality.""")

(1.4761904761904763, "Obama picks Lisa Jackson, Nancy Sutley to lead White House Council on Environmental Quality. Jackson is former head of New Jersey's environmental agency. Sutley is Los Angeles deputy mayor for energy and environment.")
(1.4285714285714286, "Obama names Lisa Jackson, Nancy Sutley to White House Council on Environmental Quality. Jackson is former head of New Jersey's environmental agency. Sutley is Los Angeles deputy mayor for energy and environment.")
(1.1363636363636365, "Obama taps Lisa Jackson and Nancy Sutley to top environmental posts. Jackson is former head of New Jersey's environmental agency. Sutley is Los Angeles deputy mayor for energy and environment.")
(1.1304347826086956, "Obama names Lisa Jackson, Nancy Sutley to top environmental posts. Jackson is former head of New Jersey's environmental agency. Sutley is Los Angeles deputy mayor for energy and environment.")
(1.0952380952380953, "Obama picks Lisa Jackson and Nancy Sutley. Jackson is former head of 

In [92]:
print_paraphrase("""CHICAGO, Illinois (CNN)  -- U.S. President-elect Barack Obama announced key members of his energy team on Monday, naming physicist Steven Chu as secretary of energy, and former EPA administrator Carol Browner to a new post in the White House to coordinate energy and climate policy.""")

(1.04, 'Obama names physicist Steven Chu as secretary of energy. Former EPA administrator Carol Browner to new post in White House. Browner will coordinate energy and climate policy in White House.')
(1.0384615384615385, 'Obama taps physicist Steven Chu as secretary of energy. Former EPA administrator Carol Browner named to new post in White House. Browner will coordinate energy and climate policy in White House.')
(1.0384615384615385, 'Obama names physicist Steven Chu as secretary of energy. Former EPA administrator Carol Browner named to new post in White House. Browner will coordinate energy and climate policy in White House.')
(1.0384615384615385, 'Obama picks physicist Steven Chu as secretary of energy. Former EPA administrator Carol Browner named to new post in White House. Browner will coordinate energy and climate policy in White House.')
(1.037037037037037, 'Obama names physicist Steven Chu as secretary of energy. Former EPA administrator Carol Browner to a new post in the Whi

In [88]:
print_paraphrase(""""Some experts have said that a lightning strike was a possibility, particularly since the plane disappeared in a storm-prone area along the equator known as the Intertropical Convergence zone (ITCZ). """)


(1.5555555555555556, 'Plane disappeared in a storm-prone area along the equator known as the Intertropical Convergence zone (ITCZ) Some experts have said that a lightning strike was a possibility, particularly since the plane disappeared.')
(1.5555555555555556, 'Plane disappeared in storm-prone area along the equator known as the Intertropical Convergence zone (ITCZ) Some experts have said that a lightning strike was a possibility, particularly since the plane disappeared.')
(1.5555555555555556, 'Plane disappeared in storm-prone area along equator known as the Intertropical Convergence zone (ITCZ) Some experts have said that a lightning strike was a possibility, particularly since the plane disappeared.')
(1.2857142857142858, 'Plane disappeared in storm-prone area along the equator known as the Intertropical Convergence zone (ITCZ) Some experts have said that a lightning strike was a possibility, particularly since the plane disappeared in a storm-prone')
(0.9393939393939394, 'Some exp

In [89]:
print_paraphrase("""This is where the trade winds of the Northern and Southern Hemispheres converge. The intense sun and warm water of the equator heats the air in the ITCZ, raising its humidity and making it buoyant. Aided by the convergence of the trade winds, the buoyant air rises, releasing the accumulated moisture in an almost constant series of thunderstorms.  The airliner's route""")

(1.12, 'The intense sun and warm water of the equator heats the air in the ITCZ, raising its humidity and making it buoyant. This is where the trade winds of the Northern and Southern Hemispheres converge.')
(1.103448275862069, 'The intense sun and warm water of the equator heats the air in the ITCZ. The buoyant air rises, releasing the accumulated moisture in an almost constant series of thunderstorms. This is where the trade winds of the Northern and Southern Hemispheres converge.')
(1.103448275862069, 'The intense sun and warm water of the equator heat the air in the ITCZ. The buoyant air rises, releasing the accumulated moisture in an almost constant series of thunderstorms. This is where the trade winds of the Northern and Southern Hemispheres converge.')
(1.09375, 'The intense sun and warm water of the equator heats the air in the ITCZ, raising its humidity. The buoyant air rises, releasing the accumulated moisture in an almost constant series of thunderstorms. This is where the 

In [72]:
from termcolor import colored
from bokeh.palettes import *
# termcolor_colors = ['red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'white']
def print_highlighted_span(text_list, span_idxs, color):
    from blessings import Terminal
    t = Terminal()
    assert isinstance(text_list, list) and len(text_list) > 0
    if not isinstance(span_idxs, list):
        span_idxs = [span_idxs]
    
    

hello world


In [84]:
print(f'All your{t.red} base {t.underline}are{t.no_underline} belong to us{t.normal}')

All your base are belong to us


In [90]:
print(t.yellow('lol'))

lol


'Who formed the universal theory of gravitation?'

[{'text': 'Isaac Newton',
  'char_spans': [[125, 136]],
  'token_spans': [[20, 21]]},
 {'text': 'Isaac Newton',
  'char_spans': [[125, 136]],
  'token_spans': [[20, 21]]},
 {'text': 'Isaac Newton',
  'char_spans': [[125, 136]],
  'token_spans': [[20, 21]]},
 {'text': 'Isaac Newton',
  'char_spans': [[125, 136]],
  'token_spans': [[20, 21]]}]